# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

10:26:38 | NewsletterAgent.newsletter_agent | INFO | Test info message
10:26:38 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
10:26:38 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

do_download=True
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251011082816006041'
    step_name = 'step_07_select_sections'
#     del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

10:26:50 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Using provided state with 106 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20251011082816006041
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20251011082816006041


In [6]:
agent.state.get_status()


{'headlines': {'total': 106},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_08_draft_sections',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': 'Categories and article counts:\ncat\nAI Cloud Infrastructure Surge     8\nAI Emerging Risks                12\nAI Investment Growth             11\nAI Startup Funding                4\nBanking AI Impact                 4\nChatGPT Bias Reduction            2\nCross-Sector AI Adoption         21\nHigh-Bandwidth Memory Growth      3\nLLM Inference Benchmarking        2\nLLM Reasoning Pretraining         2\nOpenAI Deal Network               5\nOpenAI Governance And Legal       3\nOpenAI Safety Issues              2\nOther                             4\nSam Altman AI Expansion           2\nSemantic Search Across Data       3\nSoftware Development AI Tools     4\nTech Firms AI Hiring              6\nWorkf

In [7]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


08:28:18 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting check_workflow_status
08:28:18 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Completed check_workflow_status


⏱️  Total execution time: 6.42s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Gather URLs

Step-by-step status:
- Step 1: Fetch URLs — not_started
- Step 2: Filter URLs — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Rate Articles — not_started
- Step 6: Cluster By Topic — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

Would you like me to start Step 1 (gather URLs) or resume/run all steps?


In [8]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


08:28:26 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting check_workflow_status
08:28:26 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Completed check_workflow_status
08:28:29 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting Step 1: Gather URLs
08:28:29 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Cleaning download/sources: 
08:28:29 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Successfully cleaned download/sources
08:28:29 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Cleaning download/text: 
08:28:29 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Successfully cleaned download/text
08:28:29 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Cleaning download/html: 
08:28:29 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Successfully cleaned download/html
2025-10-11 08:28:29,324 - fetcher_5644180176 - INFO - [fetcher_in

▶ Starting Step 1: step_01_fetch_urls


2025-10-11 08:28:29,456 - fetcher_5644180176 - INFO - [fetch_html] Fetching HTML from Reddit: https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day
2025-10-11 08:28:29,584 - fetcher_5644180176 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-11 08:28:29,584 - fetcher_5644180176 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-10-11 08:28:29,649 - fetcher_5644180176 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-11 08:28:29,653 - fetcher_5644180176 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-10-11 08:28:29,740 - fetcher_5644180176 - INFO - [fetch_rss] RSS fetch successful for Hacker News: 30 article

2025-10-11 08:28:31,347 - fetcher_5644180176 - INFO - scrape_url(https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNhbGllbmNlIjoibWVudGlvbiIsInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMifV0sImJ1bmRsZXMiOnsiYWxsRmVlZGx5TW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9LHsidHlwZSI6InB1YmxpY2F0aW9uQnVja2V0IiwiaWQiOiJkaXNjb3Zlcnk6YWxsLXRvcGljcyIsInRpZXIiOiJ0aWVyMSJ9XSwiY3VzdG9tTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9XSwiYm9hcmRNb2RlIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC90YWcvZ2xvYmFsLmFsbCJ9XSwiYW5ub3RhdGVkTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvdGFnL2dsb2JhbC5hbm5vdGF0ZWQifV19LCJyZWZpbmVNb2RlIjoiY3VzdG9tTW9kZSIsImxhbmd1YWdlcyI6WyJlbiJdfQ)
2025-10-11 08:28:31,347 - 

...

2025-10-11 08:28:58,860 - fetcher_5644180176 - DEBUG - Found last updated time from document.lastModified: 10/11/2025 21:28:58
2025-10-11 08:28:58,860 - fetcher_5644180176 - DEBUG - Attempting to parse last_updated: '10/11/2025 21:28:58' (type: <class 'str'>)
2025-10-11 08:28:58,860 - fetcher_5644180176 - DEBUG - Parsed datetime: 2025-10-11 21:28:58
2025-10-11 08:28:58,860 - fetcher_5644180176 - DEBUG - Added UTC timezone: 2025-10-11 21:28:58+00:00
2025-10-11 08:28:58,860 - fetcher_5644180176 - DEBUG - Converted to UTC: 2025-10-11 21:28:58+00:00
2025-10-11 08:28:58,861 - fetcher_5644180176 - DEBUG - Formatted last_updated: 2025-10-11T21:28:58Z
2025-10-11 08:28:58,861 - fetcher_5644180176 - INFO - Saving HTML to download/sources/WSJ.html
2025-10-11 08:28:58,865 - fetcher_5644180176 - INFO - [fetch_html] Parsing HTML file: download/sources/WSJ.html
2025-10-11 08:28:58,894 - fetcher_5644180176 - INFO - [fetch_html] Parsed HTML file: download/sources/WSJ.html
2025-10-11 08:28:58,894 - fetc

,source,url
0,Ars Technica,20
1,Bloomberg,31
2,Business Insider,15
3,FT,46
4,Feedly AI,70
5,Hacker News,29
6,HackerNoon,50
7,New York Times,24
8,NewsAPI,94
9,Reddit,52


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,0
1,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,1
2,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",2
3,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,3
4,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,4
...,...,...,...,...,...,...
778,NewsAPI,The AI skills gap is getting worse: Why and wh...,https://www.techtarget.com/searchhrsoftware/fe...,2025-10-10T11:27:00Z,NaN,778
779,NewsAPI,Artificial Intelligence Personalization: Moder...,https://okzest.com/blog/artificial-intelligenc...,2025-10-10T09:33:05Z,NaN,779
780,NewsAPI,"'A big mess': Expert, parents assess Alberta t...",https://www.nationalobserver.com/2025/10/10/ne...,2025-10-10T12:09:54Z,NaN,780
781,NewsAPI,Sanders plans AI tax legislation to save jobs,https://slashdot.org/submission/17341310/sande...,2025-10-10T12:21:02Z,NaN,781


08:29:09 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Completed Step 1: Gathered 887 articles
08:29:11 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting check_workflow_status
08:29:11 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Completed check_workflow_status


⏱️  Total execution time: 51.86s
📊 Final result:
Step 1 completed.

Summary:
- Fetched 887 headlines from 17 sources (RSS); 783 articles stored in persistent state.
- Workflow progress: 1/9 steps complete (11.1%).
- Next step: Step 2 — Filter URLs to AI-related content.

Would you like me to continue with Step 2 now?


In [9]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,0
1,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,1
2,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",2
3,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,3
4,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,4
...,...,...,...,...,...,...
778,NewsAPI,The AI skills gap is getting worse: Why and wh...,https://www.techtarget.com/searchhrsoftware/fe...,2025-10-10T11:27:00Z,NaN,778
779,NewsAPI,Artificial Intelligence Personalization: Moder...,https://okzest.com/blog/artificial-intelligenc...,2025-10-10T09:33:05Z,NaN,779
780,NewsAPI,"'A big mess': Expert, parents assess Alberta t...",https://www.nationalobserver.com/2025/10/10/ne...,2025-10-10T12:09:54Z,NaN,780
781,NewsAPI,Sanders plans AI tax legislation to save jobs,https://slashdot.org/submission/17341310/sande...,2025-10-10T12:21:02Z,NaN,781


In [10]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,203
8,NewsAPI,94
4,Feedly AI,70
9,Reddit,52
12,The Register,50
6,HackerNoon,50
3,FT,46
1,Bloomberg,31
5,Hacker News,29
13,The Verge,26


In [11]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:29:14 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting Step 2: Filter URLs
08:29:14 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 🔍 Filtering 783 headlines...
08:29:14 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
08:29:14 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 🔍 Filtering 783 articles for dupes.


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/tech-policy/2025/10/openai-no-longer-forced-to-save-deleted-chats-but-some-users-still-affected/
url not found - inserting new URL
checking https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/ai-models-can-acquire-backdoors-from-surprisingly-few-malicious-documents/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/vandals-deface-ads-for-ai-necklaces-that-listen-to-all-your-conversations/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/insurers-balk-at-paying-out-huge-settlements-for-claims-against-ai-firms/

url not found - inserting new URL
checking https://www.reddit.com/r/aivideo/comments/1o3ivlz/naruto_ultimate_ninja_fights/
found url, no cutoff set - treating as duplicate
checking https://www.reddit.com/r/ChatGPT/comments/1o3i2zw/do_not_use_ai_notetakers_that_join_your_calls/
url not found - inserting new URL
checking https://www.techspot.com/news/109802-fcc-could-roll-back-rule-requiring-isps-list.html
url not found - inserting new URL
checking https://www.reddit.com/r/technology/comments/1o37ldv/fcc_could_roll_back_rule_requiring_isps_to_list/
found url, no cutoff set - treating as duplicate
checking https://www.reddit.com/r/ProgrammerHumor/comments/1o30v5k/justtobesure/
found url, no cutoff set - treating as duplicate
checking https://v.redd.it/ke710278cauf1
url not found - inserting new URL
checking https://www.reddit.com/r/aivideo/comments/1o30rg3/mr_rogers_neighborhood_mandela_effect/
found url, no cutoff set - treating as duplicate
checking https://www.democracynow.org/2025/10/

08:29:14 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | URL deduplication: 473 duplicates filtered, 310 new URLs remain
08:29:14 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 🔍 Filtering 783 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient


url not found - inserting new URL
checking https://www.thestar.com.my/news/nation/2025/10/10/budget-2026-education-ministry-receives-largest-allocation-of-rm642bil
url not found - inserting new URL
checking https://www.thehindubusinessline.com/info-tech/tcs-launches-ai-hub-design-studio-in-london-to-create-5000-uk-jobs-in-3-years/article70147361.ece
url not found - inserting new URL
checking https://www.thehindubusinessline.com/info-tech/indias-sovereign-ai-model-to-be-ready-before-february-2026-meity-secretary/article70147694.ece
url not found - inserting new URL
checking https://www.livemint.com/opinion/online-views/artificial-intelligence-boom-ai-investment-2025-impact-economy-magnificent-seven-tech-companies-chatgpt-gemini-claude-11760073669001.html
url not found - inserting new URL
checking https://www.geeky-gadgets.com/awesome-self-hosted-github-projects/
url not found - inserting new URL
checking https://www.rte.ie/news/business/2025/1010/1537856-ai-stock-tech/
url not found - i

INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954
08:29:14 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Processing 13 chunks with concurrency 16


concurrency:  16


08:29:35 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Completed Step 2: 135 AI-related articles, 473 duplicates removed


⏱️  Total execution time: 21.60s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 473 duplicate URLs, classified 310 new articles, found 135 AI-related.


In [12]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

08:29:35 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting Step 3: Download Articles
08:29:36 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting concurrent scraping of 135 AI-related articles


▶ Starting Step 3: step_03_download_articles


08:29:36 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Launching browser for 135 URLs with 16 concurrent workers
08:29:36 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Worker 0 fetching 1 of 135 https://www.tomshardware.com/tech-industry/artificial-intelligence/amd-could-beat-nvidia-to-launching-ai-gpus-on-the-cutting-edge-2nm-node-instinct-mi450-is-officially-the-first-amd-gpu-to-launch-with-tsmcs-finest-tech
08:29:36 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | scrape_url(https://www.tomshardware.com/tech-industry/artificial-intelligence/amd-could-beat-nvidia-to-launching-ai-gpus-on-the-cutting-edge-2nm-node-instinct-mi450-is-officially-the-first-amd-gpu-to-launch-with-tsmcs-finest-tech)
08:29:36 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | scraping https://www.tomshardware.com/tech-industry/artificial-intelligence/amd-could-beat-nvidia-to-launching-ai-gpus-on-the-cutting-edge-2nm-node-instinct-mi450-is-officia

08:29:36 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Worker 11 fetching 11 of 135 https://www.benzinga.com/news/education/25/10/48164089/absolutely-dont-do-this-perplexity-ceo-aravind-srinivas-warns-against-misuse-of-ai-tools
08:29:36 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | scrape_url(https://www.benzinga.com/news/education/25/10/48164089/absolutely-dont-do-this-perplexity-ceo-aravind-srinivas-warns-against-misuse-of-ai-tools)
08:29:36 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | scraping https://www.benzinga.com/news/education/25/10/48164089/absolutely-dont-do-this-perplexity-ceo-aravind-srinivas-warns-against-misuse-of-ai-tools to download/html
08:29:36 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Downloading https://www.benzinga.com/news/education/25/10/48164089/absolutely-dont-do-this-perplexity-ceo-aravind-srinivas-warns-against-misuse-of-ai-tools
08:29:36 | NewsletterAgent.test_newsletter_2025101108

08:30:07 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Downloading https://www.geeky-gadgets.com/sora-app-mobile-ai-video-creation/
08:30:08 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Response: 200
08:30:08 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Saving HTML to download/html/Absolutely_Don_t_Do_This___Perplexity_CEO_Aravind_Srinivas_Warns_Against_Misuse_Of_AI_Tools.html
08:30:08 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Worker 11 completed https://www.benzinga.com/news/education/25/10/48164089/absolutely-dont-do-this-perplexity-ceo-aravind-srinivas-warns-against-misuse-of-ai-tools with status: 200
08:30:08 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Worker 11 fetching 19 of 135 https://www.theverge.com/news/798388/openai-chatgpt-political-bias-eval
08:30:08 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | scrape_url(https://www.theverge.com/news/798388/openai-chatgpt-

...

concurrency:  16


08:39:46 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Inserted 6 new site records
08:39:46 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Updated 134 URL records with final URLs


Starting with 134 rows...
Processing 134 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 126 texts...


08:39:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Completed Step 3: Downloaded 127 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 124 vs 114
    124: NewsAPI - India's sovereign AI model to be ready by February: Meity secretary
    114: NewsAPI - India’s sovereign AI model to be ready before February 2026: Meity Secretary
  Pair: 113 vs 118
    113: NewsAPI - TCS launches AI hub, design studio in London; to create 5,000 UK jobs in 3 years
    118: NewsAPI - TCS launches AI hub, design studio in UK amid layoffs, aims to create 5,000 jobs
  Pair: 55 vs 126
    55: Feedly AI - Global financial watchdogs to ramp up monitoring of AI
    126: NewsAPI - Global financial watchdogs to ramp up monitoring of AI
  Pair: 55 vs 90
    55: Feedly AI - Global financial watchdogs to ramp up monitoring of AI
    90: NewsAPI - Global Financial Watchdogs to Ramp Up Monitoring of Artificial Intelligence
  Pair: 126 vs 90
    126: NewsAPI - Global financial watchdogs to ramp up monitoring of AI
    90: NewsAPI - Global Financi

In [13]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
49,166,Feedly AI,"Developers Embrace Taskmaster, an AI Scrum Master for Code",exists,https://hackernoon.com/developers-embrace-taskmaster-an-ai-scrum-master-for-code
121,23,Bloomberg,AI’s Growth Leaves Financial Regulators Struggling to Catch Up,skipped,https://www.bloomberg.com/news/articles/2025-10-10/ai-s-growth-leaves-financial-regulators-struggling-to-catch-up
122,171,Feedly AI,I asked chatbots to find me cheap flights over the holidays. The results were mixed,403,https://www.fastcompany.com/91410840/cheap-flights-ai-find-best-deals-thanksgiving-flights-chatgpt-google-flight-imean-travel-holiday
123,118,Feedly AI,How can Anthropic stand out in the AI wars? I went to a Greenwich Village pop-up to find out.,401,https://www.marketwatch.com/story/how-can-anthropic-stand-out-in-the-ai-wars-i-went-to-a-greenwich-village-pop-up-to-find-out-e51937e5
124,629,WSJ,How Google Is Walking the AI Tightrope,skipped,https://www.wsj.com/tech/ai/google-search-ai-business-48e13735
125,630,WSJ,The Old-School Tech CEO Leading Nvidia’s Main Rival,skipped,https://www.wsj.com/tech/ceo-nvidias-main-rival-amd-lisa-su-ai-dbb18bed
126,631,WSJ,These Planes That Fly Themselves Could Transform the Skies,skipped,https://www.wsj.com/tech/ai/these-planes-that-fly-themselves-could-transform-the-skies-d5d1b7a1
127,637,WSJ,What It’s Like When Our AI Selves Are Having All the Fun,skipped,https://www.wsj.com/tech/personal-tech/i-tried-the-hot-new-ai-video-app-it-made-me-lonelier-than-ever-c9fdcceb
128,343,Techmeme,"Indian IT giant TCS cut 19,755 jobs in the quarter ended September 30, as strained US relations and a rapid shift to AI reshape India's $280B IT services sector",skipped,https://www.bloomberg.com/news/articles/2025-10-10/ai-push-drives-record-job-cuts-at-top-india-private-employer-tcs


In [14]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
121,23,Bloomberg,AI’s Growth Leaves Financial Regulators Struggling to Catch Up,skipped,https://www.bloomberg.com/news/articles/2025-10-10/ai-s-growth-leaves-financial-regulators-struggling-to-catch-up
122,171,Feedly AI,I asked chatbots to find me cheap flights over the holidays. The results were mixed,403,https://www.fastcompany.com/91410840/cheap-flights-ai-find-best-deals-thanksgiving-flights-chatgpt-google-flight-imean-travel-holiday
123,118,Feedly AI,How can Anthropic stand out in the AI wars? I went to a Greenwich Village pop-up to find out.,401,https://www.marketwatch.com/story/how-can-anthropic-stand-out-in-the-ai-wars-i-went-to-a-greenwich-village-pop-up-to-find-out-e51937e5
124,629,WSJ,How Google Is Walking the AI Tightrope,skipped,https://www.wsj.com/tech/ai/google-search-ai-business-48e13735
125,630,WSJ,The Old-School Tech CEO Leading Nvidia’s Main Rival,skipped,https://www.wsj.com/tech/ceo-nvidias-main-rival-amd-lisa-su-ai-dbb18bed
126,631,WSJ,These Planes That Fly Themselves Could Transform the Skies,skipped,https://www.wsj.com/tech/ai/these-planes-that-fly-themselves-could-transform-the-skies-d5d1b7a1
127,637,WSJ,What It’s Like When Our AI Selves Are Having All the Fun,skipped,https://www.wsj.com/tech/personal-tech/i-tried-the-hot-new-ai-video-app-it-made-me-lonelier-than-ever-c9fdcceb
128,343,Techmeme,"Indian IT giant TCS cut 19,755 jobs in the quarter ended September 30, as strained US relations and a rapid shift to AI reshape India's $280B IT services sector",skipped,https://www.bloomberg.com/news/articles/2025-10-10/ai-push-drives-record-job-cuts-at-top-india-private-employer-tcs


In [15]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

08:39:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Processing 129 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
08:39:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Using model 'gpt-4.1-mini' for summarization
08:39:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting summarization for 129 articles
08:39:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Processing 129 chunks with concurrency 16


▶ Starting Step 4: step_04_extract_summaries
concurrency:  16


08:40:08 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Extracting metadata from HTML files for 129 articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
INFO:llm:Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, system_len=1365, user_len=151
08:40:11 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Processing 129 chunks with concurrency 16


concurrency:  16
⏱️  Total execution time: 106.49s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 129/129 articles.
💾 Summaries stored in headline DataFrame.


In [16]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20, 20, 20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [17]:
# remove
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [18]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:41:34 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Rating 129 AI articles using fn_rate_articles
08:41:34 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Calculating article ratings for 129 articles
08:41:34 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Rating recency
08:41:34 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Rating spam probability
08:41:34 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Initialized LangfuseClient
08:41:34 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
08:41:34 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles
concurrency:  16


08:41:40 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | low quality articles: {0.0: 103, 1.0: 10, 5.789356328730652e-12: 1, 1.748234128271057e-13: 1, 0.18242550282051248: 1, 0.8933093855848422: 1, 0.7310585784564795: 1, 0.6513548793721767: 1, 2.9400777392844726e-11: 1, 0.9999545100305701: 1, 1.57371021068629e-11: 1, 1.7603429545652396e-06: 1, 2.289734845645553e-11: 1, 0.9999993295729128: 1, 3.701011678592672e-13: 1, 2.020683909022217e-11: 1, 4.705308323479026e-14: 1, 0.29421493708196467: 1}
08:41:40 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
08:41:40 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


concurrency:  16


08:41:44 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | on topic articles: {1.0: 89, 0.0: 5, 0.9996199233400158: 2, 0.9999998063873693: 2, 0.377540667992942: 2, 0.03732687913383872: 1, 0.9706877045032167: 1, 0.00010889693006006548: 1, 6.605216142043291e-05: 1, 0.9669140281575853: 1, 0.9999989719621736: 1, 1.3440409951135022e-08: 1, 1.2664165549094176e-14: 1, 0.9998909793854133: 1, 0.001501181950907551: 1, 0.999999448776502: 1, 0.8807969943837407: 1, 3.5517530621304715e-14: 1, 0.9890129880157539: 1, 0.3208212708736474: 1, 0.998073226003552: 1, 0.060086648833494447: 1, 0.9999723899264568: 1, 0.09534945969074979: 1, 9.056076989672867e-11: 1, 0.2227001469810028: 1, 0.0019267342527389902: 1, 3.0988191387218256e-12: 1, 0.999980972389532: 1, 0.9999978991308366: 1, 0.9975274032511579: 1, 0.9999869324773292: 1, 0.008577484930474293: 1, 0.9999993295729128: 1}
08:41:44 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Rating importance probability
08:41:44 | Newslet

...

,id,site_name,title,bradley_terry,bt_z
32,32,TechCrunch,"NPCI, which runs India's UPI, partners with OpenAI and digital payment company Razorpay on a pilot to let consumers shop and pay directly through ChatGPT",10.927173,2.275376
11,11,Business Insider,Tech CEOs marvel — and worry — about Sam Altman's dizzying race to dominate AI,10.153191,2.114209
89,89,The Economic Times,Global financial watchdogs to ramp up monitoring of AI,9.561847,1.991073
56,56,Tom's Hardware,"Amkor breaks ground on advanced chip packaging campus in Arizona, with production expected by early 2028; state officials say Amkor's investment could reach $7B",9.065240,1.887664
72,72,Entrackr,"Google plans to invest $10B to build a new 1GW data center cluster near Visakhapatnam in Andhra Pradesh, India, with operations expected to begin by July 2028",8.631295,1.797303
...,...,...,...,...,...
128,128,MarketWatch,How can Anthropic stand out in the AI wars? I went to a Greenwich Village pop-up to find out.,-9.163879,-1.908203
123,123,Reddit,It was ai all along,-9.689319,-2.017616
41,41,The Wall Street Journal,What It’s Like When Our AI Selves Are Having All the Fun,-9.979416,-2.078023
120,120,Barron's,"Stock Market Warning Signs That I’m Trying to Ignore, From AI Polyamory to a Meme Fund Rebirth",-10.072552,-2.097417


08:55:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Top 10 ids: [32, 11, 89, 56, 72, 19, 35, 64, 61, 29]
08:55:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | After round 20/26: 
08:55:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Number of ranking changes: 44
08:55:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Sum of absolute ranking changes: 1388.0 (avg rank chg 10.76)
08:55:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | last_two: 8.984496124031008, prev_two: 15.503875968992247
08:55:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | ---------------------------------------------------
08:55:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Running round 21 of max 26
08:55:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | ---------------------------------------------------
08:55:23 | NewsletterAgent.test_newsletter_20251011082816006041

,id,site_name,title,bradley_terry,bt_z
32,32,TechCrunch,"NPCI, which runs India's UPI, partners with OpenAI and digital payment company Razorpay on a pilot to let consumers shop and pay directly through ChatGPT",10.906852,2.274545
11,11,Business Insider,Tech CEOs marvel — and worry — about Sam Altman's dizzying race to dominate AI,10.132870,2.113137
89,89,The Economic Times,Global financial watchdogs to ramp up monitoring of AI,9.541521,1.989816
56,56,Tom's Hardware,"Amkor breaks ground on advanced chip packaging campus in Arizona, with production expected by early 2028; state officials say Amkor's investment could reach $7B",9.044943,1.886258
72,72,Entrackr,"Google plans to invest $10B to build a new 1GW data center cluster near Visakhapatnam in Andhra Pradesh, India, with operations expected to begin by July 2028",8.611040,1.795770
...,...,...,...,...,...
128,128,MarketWatch,How can Anthropic stand out in the AI wars? I went to a Greenwich Village pop-up to find out.,-9.171236,-1.912595
123,123,Reddit,It was ai all along,-9.696581,-2.022152
41,41,The Wall Street Journal,What It’s Like When Our AI Selves Are Having All the Fun,-9.987071,-2.082732
120,120,Barron's,"Stock Market Warning Signs That I’m Trying to Ignore, From AI Polyamory to a Meme Fund Rebirth",-10.082609,-2.102655


08:55:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Top 10 ids: [32, 11, 89, 56, 72, 19, 35, 64, 61, 29]
08:55:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | After round 21/26: 
08:55:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Number of ranking changes: 37
08:55:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Sum of absolute ranking changes: 1730.0 (avg rank chg 13.41)
08:55:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | last_two: 12.085271317829458, prev_two: 11.68217054263566
08:55:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Increase in avg rank change, stopping
08:55:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Low rated articles: 3
08:55:48 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | low rated article: Chamath Palihapitiya Says Hyperscalers Like Microsoft, Google Need To Cushion Rising Power Bills Or 'See A Lot More

⏱️  Total execution time: 854.85s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 126 articles with average rating 5.9/10.
⭐ High quality articles (≥7.0): 36
💾 Ratings stored in persistent state.


In [19]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
0,Bloomberg,"AI Jobs Shock Is Coming and Firms Aren’t Ready, Klarna CEO Says",11.419357,"Klarna CEO Sebastian Siemiatkowski warns AI will wipe out swathes of knowledge-work jobs across sectors like banking and translation; Klarna cut staff from 7,400 to about 3,000 while raising pay and plans to expand banking using its UK electronic-money licence."
2,Financial Times,How OpenAI put itself at the centre of a $1tn network of deals,11.093770,"OpenAI has positioned itself at the center of a $1 trillion network of deals by signing agreements with major tech firms, creating financial dependencies and expanding adoption of its AI technology across the industry."
11,Business Insider,Tech CEOs marvel — and worry — about Sam Altman's dizzying race to dominate AI,11.012104,"OpenAI CEO Sam Altman's rapid expansion is projected to generate over $13 billion in revenue this year but will incur $155 billion in computing costs through 2029, drawing industry acclaim for speed and concerns about consumer safety."
4,Bloomberg,"Indian IT giant TCS cut 19,755 jobs in the quarter ended September 30, as strained US relations and a rapid shift to AI reshape India's $280B IT services sector",10.258442,"Tata Consultancy Services cut 19,755 jobs in the quarter ended September 30, citing strained U.S. relations and a rapid shift to AI reshaping India's $280 billion IT services sector."
32,TechCrunch,"NPCI, which runs India's UPI, partners with OpenAI and digital payment company Razorpay on a pilot to let consumers shop and pay directly through ChatGPT",9.964417,"NPCI and Razorpay are piloting in-chat shopping and payments via ChatGPT using UPI Reserve Pay and UPI Circle, partnered with OpenAI and merchants such as BigBasket and Vi, with Google’s Gemini and Anthropic’s Claude to be included later."
...,...,...,...,...
122,HuffPost,Reba McEntire Slams Bizarre AI Rumors About Her And Dolly Parton,0.900839,Reba McEntire and Dolly Parton dismissed AI-generated fake images falsely depicting them in a hospital; McEntire condemned the misinformation and Parton reassured fans in a video.
119,Barron's,"AI Isn’t the Dot-Com Bubble, but That Won’t Stop It From Ending Badly",0.414266,Opinion piece contends AI is not a repeat of the dot-com bubble but nonetheless could end badly.
120,Barron's,"Stock Market Warning Signs That I’m Trying to Ignore, From AI Polyamory to a Meme Fund Rebirth",0.096017,"An article highlights stock-market warning signs, citing AI polyamory and a rebirth of meme funds as potential risk factors."
121,Fast Company,I asked chatbots to find me cheap flights over the holidays. The results were mixed,0.086818,"A reporter tested chatbots' ability to find cheap flights for the holidays and found mixed results in pricing accuracy, usefulness, and reliability."


In [20]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


11.4 - [AI Jobs Shock Is Coming and Firms Aren’t Ready, Klarna CEO Says](https://www.advisorperspectives.com/articles/2025/10/11/ai-jobs-shock-coming-firms-arent-ready-klarna) - Bloomberg

Klarna CEO Sebastian Siemiatkowski warns AI will wipe out swathes of knowledge-work jobs across sectors like banking and translation; Klarna cut staff from 7,400 to about 3,000 while raising pay and plans to expand banking using its UK electronic-money licence.
    


11.1 - [How OpenAI put itself at the centre of a $1tn network of deals](https://www.ft.com/content/4e39d081-ab26-4bc2-9c4c-256d766f28e2) - Financial Times

OpenAI has positioned itself at the center of a $1 trillion network of deals by signing agreements with major tech firms, creating financial dependencies and expanding adoption of its AI technology across the industry.
    


11.0 - [Tech CEOs marvel — and worry — about Sam Altman's dizzying race to dominate AI](https://www.businessinsider.com/tech-ceos-weigh-in-openai-chief-sam-altman-leadership-2025-10) - Business Insider

OpenAI CEO Sam Altman's rapid expansion is projected to generate over $13 billion in revenue this year but will incur $155 billion in computing costs through 2029, drawing industry acclaim for speed and concerns about consumer safety.
    


10.3 - [Indian IT giant TCS cut 19,755 jobs in the quarter ended September 30, as strained US relations and a rapid shift to AI reshape India's $280B IT services sector](https://www.bloomberg.com/news/articles/2025-10-10/ai-push-drives-record-job-cuts-at-top-india-private-employer-tcs) - Bloomberg

Tata Consultancy Services cut 19,755 jobs in the quarter ended September 30, citing strained U.S. relations and a rapid shift to AI reshaping India's $280 billion IT services sector.
    


10.0 - [NPCI, which runs India's UPI, partners with OpenAI and digital payment company Razorpay on a pilot to let consumers shop and pay directly through ChatGPT](https://techcrunch.com/2025/10/09/india-pilots-ai-chatbot-led-e-commerce-with-chatgpt-gemini-claude-in-the-mix/) - TechCrunch

NPCI and Razorpay are piloting in-chat shopping and payments via ChatGPT using UPI Reserve Pay and UPI Circle, partnered with OpenAI and merchants such as BigBasket and Vi, with Google’s Gemini and Anthropic’s Claude to be included later.
    

...


0.9 - [Reba McEntire Slams Bizarre AI Rumors About Her And Dolly Parton](https://www.huffpost.com/entry/reba-mcintire-ai-rumor-dolly-parton_n_68e96812e4b042e645579300) - HuffPost

Reba McEntire and Dolly Parton dismissed AI-generated fake images falsely depicting them in a hospital; McEntire condemned the misinformation and Parton reassured fans in a video.
    


0.4 - [AI Isn’t the Dot-Com Bubble, but That Won’t Stop It From Ending Badly](https://news.google.com/rss/articles/CBMifEFVX3lxTFB2QzNWQXVjUTQ2YkYwY3FtQjhKSnpIay1hMGwzaHNhaGdxeDVKTXg1Rl9fYUFXaExhcFVxd1Q4dXBFRlF5cUtxckFScUJUS2Z1ZDV5N0JMV0cxb0g0Wk55ZVN0bVBxVTJPSUZRazRqbXV3TDBmdTNPbWZDU0Q) - Barron's

Opinion piece contends AI is not a repeat of the dot-com bubble but nonetheless could end badly.
    


0.1 - [Stock Market Warning Signs That I’m Trying to Ignore, From AI Polyamory to a Meme Fund Rebirth](https://news.google.com/rss/articles/CBMimgFBVV95cUxQSWZwVkRlMGZFZUh0dnVGR1FCSC00VTNOSnZxQXM4a2hZT2NoaHBvUV9NQWQyNi1TbWhEM3FNQUNsYlNYUm5JWDlQWjE3dy1Rc1R6ZFZ2T05jLXFBTTA0YVl0RjZoYklrVVFXeXhpcEszcHEyY0ZrdUV1Z2pLa2E0eXlrT0FaUVFmR203SlJWSVdBdXJrSW9GVjBB) - Barron's

An article highlights stock-market warning signs, citing AI polyamory and a rebirth of meme funds as potential risk factors.
    


0.1 - [I asked chatbots to find me cheap flights over the holidays. The results were mixed](https://www.fastcompany.com/91410840/cheap-flights-ai-find-best-deals-thanksgiving-flights-chatgpt-google-flight-imean-travel-holiday) - Fast Company

A reporter tested chatbots' ability to find cheap flights for the holidays and found mixed results in pricing accuracy, usefulness, and reliability.
    


0.1 - [It was ai all along](https://v.redd.it/sqnaaxlr8buf1) - Reddit

A Reddit r/ChatGPT post outlines guidelines for sharing ChatGPT and DALL-E 3 content—requiring conversation links or prompts—and notes a public Discord server offering free GPT-4 with vision and image-generation tools.
    

In [21]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:55:49 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
08:55:50 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Using model 'gpt-4.1-mini' for topic extraction
08:55:50 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Processing 126 articles for topic extraction
08:55:50 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Processing 13 chunks with concurrency 16


▶ Starting Step 6: step_06_cluster_by_topic
concurrency:  16


08:55:54 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Successfully extracted 529 total topics across articles
08:55:54 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 

=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        2
n_components:       20
=== Clustering Quality Metrics ===
Number of clusters: 11
Noise points: 67 (53.2%)
Average cluster size: 5.4 ± 4.7
Cluster size range: 3 - 20
=== Quality Scores ===
Silhouette Score: 0.290 (higher is better)
Calinski-Harabasz Score: 8.7 (higher is better)
Davies-Bouldin Score: 1.083 (lower is better)
HDBSCAN Validity Index: 0.186
Composite Score: 0.238 (higher is better)

[I 2025-10-11 08:56:48,009] Trial 197 finished with value: -0.2379618049435846 and parameters: {'n_components': 20, 'min_cluster_size': 3, 'min_samples': 2}. Best is trial 74 with value: -0.2379618049435846.
=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        2
n_components:       22
=== Clustering Quality Metrics ===
Number of clusters: 10
Noise points: 64 (50.8%)
Average cluster size: 6.2 ± 5.8
Cluster size range: 3 - 23
=== Quality Scores ===
Silhouette Score: 0.241 (higher is better)
Calinski-Harabasz Sco

08:56:49 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 7: Artificial Intelligence Investment and Industry Growth
08:56:49 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | How OpenAI put itself at the centre of a $1tn network of deals (OpenAI, AI Market, Tech Partnerships, Deals, Artificial Intelligence, Funding, Gen AI)
Tech CEOs marvel — and worry — about Sam Altman's dizzying race to dominate AI (OpenAI, Sam Altman, AI Industry, Funding, Legal Issues, Enterprise Software, Artificial General Intelligence)
Mark Cuban is having 'a blast' on Sora as users generate viral videos of him (OpenAI, Sora, AI, Virtual Assistants, Deepfakes, Digital Marketing, Mark Cuban)
Barclays Says These 2 Industrial Tech Stocks Are Top Agentic AI Plays (Industrial Tech, Agentic AI, Manhattan Associates, Trimble, Supply chain optimization, Jobs And Careers, Fintech)
OpenAI's dominance is unlike anything Silicon Valley has ever seen (OpenAI, Funding, AI Dominance, Tech Partners

08:56:52 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 

08:56:52 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 4: AI Startups Secure Major Funding
08:56:52 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | An ex-OpenAI employee who helped build ChatGPT quietly launched a startup — and Mira Murati is an investor (Worktrace AI, OpenAI Alumni, AI Startups, Automation Tools, Startup Funding, Job Automation, Venture Capital)
This Brooklyn-Based AI Company Just Raised $2 Billion to Compete With DeepSeek (Open Source, AI Research, Funding, Frontier AI, Ethics, Governance, Language Models)
ARC Group Invests in ScanTech AI Systems, Backing Company’s AI-Powered Security and Regulatory Alignment (Funding, AI Security, ScanTech AI Systems Inc., Aviation, Logistics, Nasdaq:STAI, Policy And Regulation)
Govini, a Palantir competitor whose AI software Ark helps militaries manage product lifecycles, raised $150M from Bain Capital and says it hit $100M 

⏱️  Total execution time: 67.03s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 126 articles into topic clusters.


In [22]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:56:57 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179


▶ Starting Step 7: step_07_select_sections


08:57:44 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Cleaning up initial categories: ['OpenAI Product Safety Issues', 'Indian IT Job Cuts', 'Reflection AI $2B Raise', 'AI-Powered Personalization in Business', "Sam Altman's AI Expansion", 'Data Center Energy Management', 'LLM Reasoning Pretraining', 'AI Impact on Jobs and Banking', 'AI Tools Transforming Software Development', 'Copilot Adoption Monitoring', 'AI Cloud Infrastructure Surge', 'Taskmaster AI Scrum Master', "Reducing Bias in OpenAI's ChatGPT", 'AI Adoption Across Sectors and Regions', 'LLM Inference Benchmarking', 'OpenAI Governance And Legal', 'AMD 2nm AI GPUs', 'Tech Firms Expanding AI and Jobs', 'AI Transformation Impact on Workplaces', 'OpenAI Deal Network', 'High-Bandwidth Memory Growth', 'Artificial Intelligence Investment and Industry Growth', 'Campus Delivery Robots Expansion', 'AI Workforce Disruption', 'AI Startups Secure Major Funding', 'Other', 'AI In-Chat Payments Pilot', 'Global Financial AI

08:59:04 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Handling singleton categories
08:59:04 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Singleton categories (1): ['Global Financial AI Oversight']
08:59:04 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Non-singleton categories (19): ['Cross-Sector AI Adoption', 'AI Emerging Risks', 'AI Investment Growth', 'AI Cloud Infrastructure Surge', 'Workforce AI Impact', 'Tech Firms AI Hiring', 'OpenAI Deal Network', 'Other', 'Software Development AI Tools', 'AI Startup Funding', 'High-Bandwidth Memory Growth', 'OpenAI Governance And Legal', 'Semantic Search Across Data', 'Banking AI Impact', 'Sam Altman AI Expansion', 'ChatGPT Bias Reduction', 'LLM Inference Benchmarking', 'OpenAI Safety Issues', 'LLM Reasoning Pretraining']
08:59:04 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Reassigning 1 singleton articles to non-singleton categories
08:59:08 | NewsletterAgent.test_ne

⏱️  Total execution time: 131.17s
📊 Final result:
Categories and article counts:
cat
AI Cloud Infrastructure Surge     8
AI Emerging Risks                12
AI Investment Growth             11
AI Startup Funding                4
Banking AI Impact                 4
ChatGPT Bias Reduction            2
Cross-Sector AI Adoption         21
High-Bandwidth Memory Growth      3
LLM Inference Benchmarking        2
LLM Reasoning Pretraining         2
OpenAI Deal Network               5
OpenAI Governance And Legal       3
OpenAI Safety Issues              2
Other                             4
Sam Altman AI Expansion           2
Semantic Search Across Data       3
Software Development AI Tools     4
Tech Firms AI Hiring              6
Workforce AI Impact               8


In [7]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:27:17 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Step 8a: Selecting stories from 106 total
10:27:17 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 14 must-include stories with rating > 8
10:27:17 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | 92 stories with rating <= 8


▶ Starting Step 8: step_08_draft_sections


10:27:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Selected 86 diverse Tier 2 stories via MMR
10:27:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Total selected stories: 100 (target: 100)
10:27:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Step 8b: Drafting sections for 18 categories from selected stories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1860, user_len=22
10:27:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Initialized LLMagent:
system_prompt: You are a newsletter editor transforming a collection of raw news stories into a compelling, coherent topic summary.

# TASK
Transform the list of news stories into a well-structured newsletter section with a strong title, crisp headlines, and punchy summaries.

# INPUT
- a list of json objects sorted by rating

10:27:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | User message: STORIES:
[{"rating":10.0743622384,"summary":"OpenAI says GPT-5, after an internal stress test across 100 divisive topics, shows a 30% reduction in political bias versus earlier versions; it resists highly charged prompts better and added user-adjustable tones and public behavior guidelines amid political pressure.","site_name":"The Verge","url":"https:\/\/www.theverge.com\/news\/798388\/openai-chatgpt-political-bias-eval"},{"rating":6.5196450257,"summary":"OpenAI says GPT-5 powering ChatGPT cuts political bias by 30%, with under 0.01% of responses showing political slant; independent research, however, finds persistent AI biases\u2014caste, gender, beauty and non\u2011English educational harms\u2014across other models.","site_name":"Digital Trends","url":"https:\/\/www.digitaltrends.com\/computing\/openai-says-chatgpt-is-the-least-biased-it-has-ever-been-but-its-not-all-roses\/"}]
10:27:23 | Newslett

10:27:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | User message: STORIES:
[{"rating":8.0229878524,"summary":"Nvidia developed a reinforcement-learning pretraining (RLP) method that trains LLMs to generate reasoning chains before token prediction, boosting reasoning-benchmark performance by up to 17% and improving downstream fine-tuning reliability for enterprise workflows.","site_name":"VentureBeat","url":"https:\/\/venturebeat.com\/ai\/nvidia-researchers-boost-llms-reasoning-skills-by-getting-them-to-think"},{"rating":7.0516591299,"summary":"Google DeepMind introduced Gemini Robotics 1.5 and Gemini Robotics-ER 1.5, models that enable robots such as Aloha 2 to perform complex multistep tasks with advanced perception, spatial reasoning and natural-language explanations, and to use tools like Google Search.","site_name":"Live Science","url":"https:\/\/www.livescience.com\/technology\/robotics\/robots-receive-major-intelligence-boost-thanks-to-google-deepminds-thinki

...

10:27:23 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | User message: STORIES:
[{"rating":6.6831775951,"summary":"Former OpenAI product manager Angela Jiang launched Worktrace AI to automate repetitive enterprise tasks by observing employees; investors include Mira Murati, OpenAI's startup fund, 8VC and Conviction, and it is raising a $10 million seed at a $50 million valuation.","site_name":"Business Insider","url":"https:\/\/www.businessinsider.com\/former-openai-product-manager-launches-worktrace-ai-mira-murati-backing-2025-10"},{"rating":6.584617738,"summary":"Govini says it surpassed $100 million in annual recurring revenue and secured a $150 million investment from Bain Capital to expand its AI Ark product and team for military supply-chain and product-lifecycle modernization.","site_name":"CNBC","url":"https:\/\/www.cnbc.com\/2025\/10\/10\/defense-tech-govini-palantir-revenue.html"},{"rating":5.6232630786,"summary":"ARC Group International provided ScanTech AI S

10:27:43 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Result: ## AI Supercharges Search and Research

- ThalamusDB brings semantic SQL to DuckDB, using LLMs to query text, images, audio and stream approximate results with error bounds - [GitHub Gist](https://github.com/itrummer/thalamusdb)
- AI research tools like Scite, Semantic Scholar, Elicit, and Trinka streamline literature search, writing, and data analysis for faster academic workflows - [Geeky Gadgets](https://www.geeky-gadgets.com/ai-research-tools-2025/)
- Mixedbread Search enables natural-language queries across 50,000+ National Gallery images for intuitive discovery by artist and subject - [Mixed Bread](https://nga.demo.mixedbread.com/)
10:27:43 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Result: ## GPT-5 Bias Check

- OpenAI says GPT-5 cuts political bias 30% with <0.01% slanted replies; adds tone controls and behavior rules as studies still flag wider AI biases. - [The Verge](https:/

10:28:00 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Result: ## AI Reasoning Leaps Forward

- Nvidia's RLP trains LLMs to think first, boosting reasoning benchmarks up to 17% and improving enterprise fine-tuning reliability. - [VentureBeat](https://venturebeat.com/ai/nvidia-researchers-boost-llms-reasoning-skills-by-getting-them-to-think)
- DeepMind's Gemini Robotics 1.5 enables robots like Aloha 2 to perform multistep tasks with spatial reasoning, natural-language explanations, and tool use. - [Live Science](https://www.livescience.com/technology/robotics/robots-receive-major-intelligence-boost-thanks-to-google-deepminds-thinking-ai-a-pair-of-models-that-help-machines-understand-the-world)
10:28:06 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Result: ## AI Bets and Builds

- EU Commission launches six AI factories across Czechia, Lithuania, Netherlands, Romania, Spain and Poland - [Biztoc](https://biztoc.com/x/04127493ca266921)
- Malaysia commits

10:28:47 | NewsletterAgent.test_newsletter_20251011082816006041 | INFO | Result: ## AI's Job Reckoning

- BSI: 39% cut entry-level roles for AI; 43% plan more, 95% see no ROI, risking future leaders without upskilling. - [The Register](https://www.theregister.com/2025/10/10/ai_is_displacing_entrylevel_professionals/)
- Klarna CEO: AI will wipe out many knowledge jobs; staff cut from 7,400 to 3,000 while pay rose; banking expansion via UK e-money license. - [Bloomberg](https://mercury.bloomberg.com/news/T3UYZQGOT0JK)
- AI displaces translators, reshaping language work as employers push automation; linguists report emotional strain and forced adaptation. - [Financial Times](https://www.ft.com/content/50b1f03e-1d10-4a7c-afa3-b48e9a8d5133)
- Google Cloud CEO: AI augments jobs; 10% engineer productivity gain, more hiring planned; AI tools ease customer service workloads without layoffs. - [Business Insider](https://www.businessinsider.com/google-cloud-ceo-ai-automate-jobs-2025-10)
- AI skil

✅ Completed Step 8: Drafted 18 sections with 2019 total words
⏱️  Total execution time: 148.09s
📊 Final result:
Drafted 18 sections with 2019 total words


In [10]:
for v in state.newsletter_section_text.values():
    display(Markdown(v))


## AI's Job Reckoning

- BSI: 39% cut entry-level roles for AI; 43% plan more, 95% see no ROI, risking future leaders without upskilling. - [The Register](https://www.theregister.com/2025/10/10/ai_is_displacing_entrylevel_professionals/)
- Klarna CEO: AI will wipe out many knowledge jobs; staff cut from 7,400 to 3,000 while pay rose; banking expansion via UK e-money license. - [Bloomberg](https://mercury.bloomberg.com/news/T3UYZQGOT0JK)
- AI displaces translators, reshaping language work as employers push automation; linguists report emotional strain and forced adaptation. - [Financial Times](https://www.ft.com/content/50b1f03e-1d10-4a7c-afa3-b48e9a8d5133)
- Google Cloud CEO: AI augments jobs; 10% engineer productivity gain, more hiring planned; AI tools ease customer service workloads without layoffs. - [Business Insider](https://www.businessinsider.com/google-cloud-ceo-ai-automate-jobs-2025-10)
- AI skills gap widens; nearly half of employees lack training and mishandle sensitive data, urging critical thinking, leader-led upskilling, and risk-benefit assessments. - [TechTarget](https://www.techtarget.com/searchhrsoftware/feature/The-AI-skills-gap-is-getting-worse-Why-and-what-to-do)
- Sen. Bernie Sanders to propose robot tax on corporations replacing workers with AI or automation, to offset lost tax revenue and discourage displacement. - [Slashdot Linux](https://slashdot.org/submission/17341310/sanders-plans-ai-tax-legislation-to-save-jobs)
- Microsoft unveils $4B, five-year Elevate Washington to deliver free AI tools and training to all K-12 districts and colleges, targeting 25 million learners by 2026. - [Slashdot Linux](https://news.slashdot.org/story/25/10/11/0012220/microsoft-to-provide-free-ai-tools-for-washington-state-schools)

## OpenAI's Power Plays

- OpenAI at center of $1 trillion industry deal network, deepening dependencies and adoption - [Financial Times](https://www.ft.com/content/4e39d081-ab26-4bc2-9c4c-256d766f28e2)
- OpenAI buys 10% of AMD in multibillion-dollar deal; AMD shares surge 34%+ - [Marketplace](https://www.marketplace.org/episode/2025/10/10/openai-partners-with-amd-google-home-devices-get-ai-update)
- OpenAI valued at $500B, stays private; investors seek exposure via Microsoft, Nvidia, or DXYZ as Sora and AgentKit launch - [247wallst](https://247wallst.com/investing/2025/10/11/invest-here-to-get-a-piece-of-openai/)
- Duolingo allegedly tops OpenAI token consumption list; app reports 34.2M DAUs and 9.5M subscribers in 2024 - [Wccftech](https://wccftech.com/duolingo-allegedly-tops-a-list-of-openais-top-30-customers-by-token-consumption/)
- Mark Cuban okays Sora 2 deepfakes featuring Cost Plus Drugs ads; viral AI videos leverage his likeness - [Business Insider](https://www.businessinsider.com/mark-cuban-free-advertisements-for-cost-plus-drugs-sora-2-2025-10)

## GPT-5 Bias Check

- OpenAI says GPT-5 cuts political bias 30% with <0.01% slanted replies; adds tone controls and behavior rules as studies still flag wider AI biases. - [The Verge](https://www.theverge.com/news/798388/openai-chatgpt-political-bias-eval) [Digital Trends](https://www.digitaltrends.com/computing/openai-says-chatgpt-is-the-least-biased-it-has-ever-been-but-its-not-all-roses/)

## OpenAI Under Scrutiny

- OpenAI subpoenas AI-reg advocates amid Elon Musk countersuit, seeking messages on alleged Musk funding; includes Encode AI lawyer Nathan Calvin - [The Verge](https://www.theverge.com/news/798523/openai-ai-regulation-advocates-subpoenas-police)
- Court lifts preservation order; OpenAI can stop saving most deleted ChatGPT logs as of Sept 26, some deleted chats still monitored - [Ars Technica](https://arstechnica.com/tech-policy/2025/10/openai-no-longer-forced-to-save-deleted-chats-but-some-users-still-affected/)
- Guardian columnist criticizes Altman’s Sora 2 over alleged copyright misuse; OpenAI’s Nvidia/AMD ties help drive $1T+ deals this year - [The Guardian](https://www.theguardian.com/commentisfree/2025/oct/10/sam-altman-copyright-ai-sora-2-video-generator)

## OpenAI: Scale and Sora

- OpenAI speeds ahead: $13B revenue this year, $155B compute costs by 2029, raising industry acclaim and consumer safety concerns - [Business Insider](https://www.businessinsider.com/tech-ceos-weigh-in-openai-chief-sam-altman-leadership-2025-10)
- Sora mobile app launches: text-to-video with customizable Cameos avatars, edit/remix tools, watermarked downloads, social feeds, DMs, privacy controls, multiple orientations - [Geeky Gadgets](https://www.geeky-gadgets.com/sora-app-mobile-ai-video-creation/)

...

## Bots That Build Software

- Anthropic launches Claude Code 'Superpowers' for autonomous planning, TDD, parallel git worktrees, skills, delegation; sharing and memory coming. - [FSCK](https://blog.fsck.com/2025/10/09/superpowers/)
- Developer live-prototypes with Claude Code at Denver meetup, using crowd prompts, voice commands, and a 'fuzzy compiler' to improvise a basket-weaving software cursor. - [Hacker Noon](https://hackernoon.com/can-you-improvise-software-one-developer-put-ai-to-the-test)
- Bitrig turns iPhone prompts or voice into simple apps like to-do lists and workout trackers, but many generated UIs misfire with unresponsive buttons. - [Tom's Guide](https://www.tomsguide.com/ai/i-tried-an-app-that-makes-apps-using-ai-on-my-iphone-and-it-felt-like-the-future)

## AI Bets and Builds

- EU Commission launches six AI factories across Czechia, Lithuania, Netherlands, Romania, Spain and Poland - [Biztoc](https://biztoc.com/x/04127493ca266921)
- Malaysia commits RM5.9b and RM2b Sovereign AI Cloud toward 2030 AI push; tax breaks and National AI Office funding - [The Star](https://www.thestar.com.my/tech/tech-news/2025/10/10/anwar-rm59bil-cross-ministry-allocation-to-keep-m039sia-at-forefront-of-ai-development)
- U.S. AI diagnostics market to grow from $424M in 2024 to $1.83B by 2033, 17.68% CAGR - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/10/3164756/28124/en/United-States-Artificial-Intelligence-in-Diagnostics-Market-Research-Report-2025-2033-Precision-Medicine-Advanced-Imaging-Analytics-and-AI-Enabled-Clinical-Decision-Support-Spur-Ex.html)
- Nvidia seeks alternative Taiwan HQ site after Beitou-Shilin project stalls over surface rights; government mulls regulatory fix - [The Economic Times](https://m.economictimes.com/tech/tech-bytes/nvidia-seeks-alternative-sites-for-its-new-taiwan-headquarters/articleshow/124453187.cms)
- Oracle’s AI alliances lift OCI revenue 53% and backlog to $455B; Palantir offers faster growth with higher valuation and concentration risk - [Forbes](https://www.forbes.com/sites/investor-hub/article/is-oracle-or-palantir-the-smarter-ai-stock-to-buy/)
- Nvidia posts $26.4B net income and $56.8B cash as CoreWeave revenue quadruples but remains unprofitable - [The Motley Fool](https://www.fool.com/investing/2025/10/10/better-ai-stock-to-buy-coreweave-or-nvidia/)
- Ken Griffin’s Citadel dumps over 80% of Broadcom, boosts Nvidia to 8M+ shares after 90% rebound - [The Motley Fool](https://www.fool.com/investing/2025/10/11/billionaire-ken-griffin-citadel-broadcom-stock-ai/)
- Berkshire Hathaway concentrates 24% of $304B portfolio in Apple and Amazon, two AI-driven growth plays - [The Motley Fool](https://www.fool.com/investing/2025/10/10/warren-buffett-ai-stocks-berkshire-hathaway/)
- Three AI ETFs to watch: AIQ, BAI, and ROBO, with fees from 0.55% to 0.95% - [The Motley Fool](https://www.fool.com/investing/2025/10/10/3-ai-etfs-to-buy-now-for-the-coming-tech-revolutio/)

## AI Funding Frenzy

- Govini tops $100M ARR, lands $150M from Bain to scale AI Ark for military supply-chain and lifecycle modernization - [CNBC](https://www.cnbc.com/2025/10/10/defense-tech-govini-palantir-revenue.html)
- ScanTech AI secures up to $50M equity line, including $500K PIPE, to advance SENTINEL AI, Nasdaq compliance, and 2026 international certification - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/10/3164812/0/en/ARC-Group-Invests-in-ScanTech-AI-Systems-Backing-Company-s-AI-Powered-Security-and-Regulatory-Alignment.html)
- Prezent raises $30M at $400M valuation to acquire AI services firms, starting with life-sciences presentation company Prezentium; total funding now $74M+ - [TechCrunch](https://techcrunch.com/2025/10/10/prezent-raises-30-million-to-acquire-ai-services-firms-starting-with-founders-other-company/)
- Ex-OpenAI PM launches Worktrace AI to automate enterprise tasks; backed by Mira Murati, OpenAI's fund, 8VC, Conviction; raising $10M seed at $50M valuation - [Business Insider](https://www.businessinsider.com/former-openai-product-manager-launches-worktrace-ai-mira-murati-backing-2025-10)

## AI Boom, Backlash, and Boundaries

- Top financiers warn AI bubble could burst within 6–24 months, with risk of US‑Europe contagion and openings for European tech. - [RTÉ](https://www.rte.ie/news/business/2025/1010/1537856-ai-stock-tech/)
- Ex-Twitter/Meta policy lead: AI repeats social-media governance mistakes, needs regulation, cross-sector testing, and global standards as energy use and critical integration grow. - [Fortune](https://fortune.com/2025/10/11/front-row-seat-social-media-revolution-same-mistakes-ai/)
- Google walks tightrope between AI innovation, safety, regulation, and business pressure in Search. - [The Wall Street Journal](https://www.wsj.com/tech/ai/google-search-ai-business-48e13735)
- You can disable many AI features in Google Search and Workspace, but not switch AI off entirely. - [CNET](https://www.cnet.com/tech/services-and-software/how-to-turn-off-or-reduce-ai-in-google-apps-and-services/)
- 82% of US CEOs see AI as essential, but only 41% align on product vision; experts urge differentiation via unique data and customer insight. - [Forbes](https://www.forbes.com/sites/annegriffin/2025/10/10/when-ai-undermines-product-strategy-and-what-to-do-instead/)
- AI meeting notetakers with calendar access can auto-join calls without consent; some firms ban them, favoring non-joining or native transcription tools. - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1o3i2zw/do_not_use_ai_notetakers_that_join_your_calls/)
- Perplexity’s $200 Comet agent browser can ace Coursera and certification tests; CEO warns against misuse, notes prompt-injection vulnerabilities. - [Benzinga](https://www.benzinga.com/news/education/25/10/48164089/absolutely-dont-do-this-perplexity-ceo-aravind-srinivas-warns-against-misuse-of-ai-tools)
- AI-generated ‘homeless man’ TikTok prank triggers false 911 calls in US and UK, wasting resources and dehumanizing vulnerable people. - [Gizmodo](https://gizmodo.com/police-say-people-keep-calling-911-over-an-ai-homeless-man-tiktok-prank-2000671130)
- Ex–UK PM Rishi Sunak takes advisory roles at Microsoft and Anthropic, pledges pay to charity, raising fresh ethics questions. - [International Business Times](https://www.ibtimes.co.uk/ex-pm-rishi-sunak-lands-shocking-new-job-microsoft-ai-giant-anthropic-1747272)
- AI excels in narrow tasks but lacks AGI, robust common sense, and sustainability; human-AI collaboration and energy-efficient advances remain critical. - [Geeky Gadgets](https://www.geeky-gadgets.com/ai-limitations-vs-human-intelligence/)
- Parents weigh AI’s creative benefits against privacy, safety, and ethics as they raise kids in the AI era. - [Business Insider](https://www.businessinsider.com/ai-workshop-parenting-new-generation-kids-technology-chatgpt-gemini-exposure-2025-10)

## AI Supercharges Search and Research

- ThalamusDB brings semantic SQL to DuckDB, using LLMs to query text, images, audio and stream approximate results with error bounds - [GitHub Gist](https://github.com/itrummer/thalamusdb)
- AI research tools like Scite, Semantic Scholar, Elicit, and Trinka streamline literature search, writing, and data analysis for faster academic workflows - [Geeky Gadgets](https://www.geeky-gadgets.com/ai-research-tools-2025/)
- Mixedbread Search enables natural-language queries across 50,000+ National Gallery images for intuitive discovery by artist and subject - [Mixed Bread](https://nga.demo.mixedbread.com/)

In [28]:
must_include = headline_df[headline_df['rating'] > 8.0].copy()
logger.info(f"Tier 1 (rating > 8.0): {len(must_include)} must-include stories")


09:36:41 | NewsletterAgent.newsletter_agent | INFO | Tier 1 (rating > 8.0): 14 must-include stories


In [30]:
candidates = headline_df.loc[headline_df['rating'] <= 8.0].copy()
len(candidates)

92

In [43]:
candidates_for_embed = candidates.copy()

candidates_for_embed['short_summary'] = candidates_for_embed['short_summary'].fillna('')
candidates_for_embed.loc[candidates_for_embed['short_summary'] == '', 'short_summary'] = candidates_for_embed['summary'].fillna('')
candidates_for_embed.loc[candidates_for_embed['short_summary'] == '', 'short_summary'] = candidates_for_embed['title'].fillna('')


In [44]:
                embedding_df = await _get_embeddings_df(candidates_for_embed)


In [45]:
                    embeddings = embedding_df.values


In [46]:
target_tier2 = 100 - len(must_include)
target_tier2

86

In [49]:
from sklearn.metrics.pairwise import cosine_similarity

def mmr_selection(
    df: pd.DataFrame,
    embeddings: np.ndarray,
    n: int = 100,
    lambda_param: float = 0.5
) -> pd.DataFrame:
    """
    Select diverse, high-quality stories using Max Marginal Relevance.

    Balances story quality (rating) with diversity (embedding similarity)
    to avoid redundant coverage of the same story angles.

    Args:
        df: DataFrame with 'rating' column and matching embedding indices
        embeddings: numpy array of embeddings (shape: [len(df), embedding_dim])
        n: Number of stories to select
        lambda_param: Tradeoff between relevance (rating) and diversity
                     1.0 = pure rating, 0.0 = pure diversity, 0.5 = balanced

    Returns:
        DataFrame subset of n selected stories with maximum rating and diversity
    """
    if len(df) <= n:
        return df

    # Normalize ratings to 0-1 scale
    max_rating = df['rating'].max()
    min_rating = df['rating'].min()
    rating_range = max_rating - min_rating

    if rating_range == 0:
        normalized_ratings = pd.Series([0.5] * len(df), index=df.index)
    else:
        normalized_ratings = (df['rating'] - min_rating) / rating_range

    selected_indices = []
    remaining_indices = list(df.index)

    # Start with highest rated story
    first_idx = df['rating'].idxmax()
    selected_indices.append(first_idx)
    remaining_indices.remove(first_idx)

    # Get embedding index mapping (df.index might not be 0..n-1)
    idx_to_embedding_pos = {idx: pos for pos, idx in enumerate(df.index)}

    while len(selected_indices) < n and remaining_indices:
        mmr_scores = []

        for idx in remaining_indices:
            # Relevance: normalized rating
            relevance = normalized_ratings.loc[idx]

            # Diversity: 1 - max_similarity to already selected
            embedding_pos = idx_to_embedding_pos[idx]
            selected_positions = [idx_to_embedding_pos[i]
                                  for i in selected_indices]

            similarities = cosine_similarity(
                embeddings[embedding_pos:embedding_pos+1],
                embeddings[selected_positions]
            )
            max_similarity = similarities.max()
            diversity = 1 - max_similarity

            # Combined MMR score
            mmr = lambda_param * relevance + (1 - lambda_param) * diversity
            mmr_scores.append((idx, mmr))

        # Select highest MMR
        best_idx = max(mmr_scores, key=lambda x: x[1])[0]
        selected_indices.append(best_idx)
        remaining_indices.remove(best_idx)

    return df.loc[selected_indices]


In [50]:
                    tier2_selected = mmr_selection(
                        df=candidates,
                        embeddings=embeddings,
                        n=target_tier2,
                        lambda_param=0.5  # 50% rating, 50% diversity
                    )


In [56]:
type(must_include)



pandas.core.frame.DataFrame

In [57]:

# Combine tiers
selected_df = pd.concat(
    [must_include, tier2_selected])
logger.info(
    f"Total selected stories: {len(selected_df)} (target: ~100-120)")
selected_df

10:12:24 | NewsletterAgent.newsletter_agent | INFO | Total selected stories: 100 (target: ~100-120)


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,...,bt_z,adjusted_len,rating,bradley_terry,topics,extended_summary,cluster,cluster_name,cat,dupe_id
0,Feedly AI,AI Jobs Shock Is Coming and Firms Aren’t Ready...,https://www.advisorperspectives.com/articles/2...,NaN,NaN,0,True,200,https://mercury.bloomberg.com/news/T3UYZQGOT0JK,download/html/AI_Jobs_Shock_Is_Coming_and_Firm...,...,2.863779,0.555578,11.419357,1.962121,"[AI Job Disruption, Job Automation, Fintech, B...",AI Jobs Shock Is Coming and Firms Aren’t Ready...,5,AI Impact on Jobs and Banking,Workforce AI Impact,-1.0
1,FT,‘They wanted me to make myself obsolete’: tran...,https://www.ft.com/content/50b1f03e-1d10-4a7c-...,NaN,NaN,1,True,200,https://www.ft.com/content/50b1f03e-1d10-4a7c-...,download/html/They_wanted_me_to_make_myself_ob...,...,0.970813,0.137671,8.545375,-4.089286,"[Job Displacement, Translation, Professional C...",‘They wanted me to make myself obsolete’: tran...,-1,Other,Workforce AI Impact,-1.0
2,FT,How OpenAI put itself at the centre of a $1tn ...,https://www.ft.com/content/4e39d081-ab26-4bc2-...,NaN,NaN,2,True,200,https://www.ft.com/content/4e39d081-ab26-4bc2-...,download/html/How_OpenAI_put_itself_at_the_cen...,...,3.786675,0.141136,12.093770,4.912421,"[OpenAI, AI Market, Tech Partnerships, Deals, ...",How OpenAI put itself at the centre of a $1tn ...,7,Artificial Intelligence Investment and Industr...,OpenAI Deal Network,-1.0
4,The Register,Managers are throwing entry-level workers unde...,https://go.theregister.com/feed/www.theregiste...,2025-10-10T19:49:20.00Z,"<h4>Does it work? Inconclusive. Still, 55% of ...",5,True,200,https://www.theregister.com/2025/10/10/ai_is_d...,download/html/Managers_are_throwing_entry-leve...,...,2.787590,0.811173,8.827390,1.718560,"[Job Automation, Entry-Level Jobs, Skill Devel...",Managers are throwing entry-level workers unde...,10,AI Transformation Impact on Workplaces,Workforce AI Impact,-1.0
5,The Verge,OpenAI is trying to clamp down on ‘bias’ in Ch...,https://www.theverge.com/news/798388/openai-ch...,NaN,NaN,6,True,200,https://www.theverge.com/news/798388/openai-ch...,download/html/OpenAI_is_trying_to_clamp_down_o...,...,3.155340,0.677333,10.074362,2.894178,"[GPT-5, Political Bias, Bias And Fairness, Mod...",OpenAI is trying to clamp down on ‘bias’ in Ch...,2,Reducing Bias in OpenAI's ChatGPT,ChatGPT Bias Reduction,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Hacker News,Show HN: Semantic search over the National Gal...,https://nga.demo.mixedbread.com/,"Fri, 10 Oct 2025 20:33:25 +0000","<a href=""https://news.ycombinator.com/item?id=...",111,True,200,https://nga.demo.mixedbread.com/,download/html/Show_HN__Semantic_search_over_th...,...,-0.014140,0.000000,1.985751,-7.237971,"[National Gallery, Art Archives, Semantic Sear...",Show HN: Semantic search over the National Gal...,0,Semantic Search Across Diverse Data,Semantic Search Across Data,-1.0
73,The Guardian,Using a swearword in your Google search can st...,https://www.theguardian.com/technology/2025/oc...,"Fri, 10 Oct 2025 23:00:07 GMT",<p>Artificial intelligence is more than Trump ...,90,True,200,https://www.theguardian.com/technology/2025/oc...,download/html/Using_a_swearword_in_your_Google...,...,0.770028,0.332438,3.438281,-4.731155,"[AI Applications, Language Models, Ethics, Pri...",Using a swearword in your Google search can st...,-1,Other,Cross-Sector AI Adoption,-1.0
104,Feedly AI,3 AI ETFs to Buy Now for the Coming Tech Revol...,https://www.fool.com/investing/2025/10/10/3-ai...,NaN,NaN,124,True,200,https://www.fool.com/investing/2025/10/10/3-ai...,download/html/3_AI_ETFs_to_Buy_Now_for_the_Com...,...,1.850994,0.796644,2.539890,-1.275538,"[AI ETFs, Investing, Robotics, Job Automation,...",3 AI ETFs to Buy Now for the Coming Tech Revol...,7,Artificial Intelligence Investment and Industr...,AI Investment Growth,-1.0
75,NewsAPI,Rishi Sunak Lands Shocking New Job With Micros...,https://w

In [58]:
            headline_df = selected_df.copy()


In [59]:
            # Get unique categories
            categories = headline_df['cat'].unique().tolist()
            categories = [cat for cat in categories if cat != "Other"]


In [62]:
class SectionStoryLink(BaseModel):
    url: str = Field(description="URL of the article")
    site_name: str = Field(description="Name of the website/source")

    def __str__(self):
        return f"[{self.site_name}]({self.url})"


class SectionStory(BaseModel):
    headline: str = Field(description="Summary of the story")
    links: List[SectionStoryLink] = Field(
        description="List of links related to this story")
    prune: bool = Field(description="Whether to prune/exclude this story")

    def __str__(self):
        return f"- {self.headline} - " + " ".join([str(s) for s in self.links])


class Section(BaseModel):
    section_title: str = Field(description="Title of the newsletter section")
    headlines: List[SectionStory] = Field(
        description="List of stories in this section")

    def __str__(self):
        return f"## {self.section_title}\n\n" + "\n".join(
            [str(h) for h in self.headlines if not h.prune]
        )



In [64]:
            write_section_system_prompt, write_section_user_prompt, model = \
                LangfuseClient().get_prompt("newsagent/write_section")

            write_section_agent = LLMagent(
                system_prompt=write_section_system_prompt,
                user_prompt=write_section_user_prompt,
                output_type=Section,
                model=model,
                verbose=True,
                logger=logger
            )


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1860, user_len=22
10:15:35 | NewsletterAgent.newsletter_agent | INFO | Initialized LLMagent:
system_prompt: You are a newsletter editor transforming a collection of raw news stories into a compelling, coherent topic summary.

# TASK
Transform the list of news stories into a well-structured newsletter section with a strong title, crisp headlines, and punchy summaries.

# INPUT
- a list of json objects sorted by rating (highest first)
- Each item: { "rating": number, "short_summary": string, "site_name": string, "url": string }

# OUTPUT
- minified JSON only, in the specified schema, with section title and list of headlines, each with a list of links
- no code fences, no line breaks, no extra whitespace).

# WORKFLOW
1. **Section title**
- Infer the dominant, unifying topic of the set of stories (by co

In [65]:
            async def draft_section(cat, agent):
                """Draft a section for a given category"""
                # Get articles for this category, sorted by rating, convert to JSON
                cat_df = headline_df.loc[headline_df["cat"] == cat].sort_values(
                    "rating", ascending=False)

                input_text = cat_df[["rating", "short_summary", "site_name", "final_url"]].rename(columns={"short_summary": "summary", "final_url": "url"}).to_json(
                    orient="records")

                # Call the LLM to draft the section
                response = await agent.run_prompt(input_text=input_text)

                return (cat, response)

In [66]:
            # Draft all sections asynchronously
            draft_tasks = [draft_section(cat, write_section_agent)
                           for cat in categories]
            draft_results = await asyncio.gather(*draft_tasks, return_exceptions=True)


10:16:13 | NewsletterAgent.newsletter_agent | INFO | User message: STORIES:
[{"rating":11.419357322,"summary":"Klarna CEO Sebastian Siemiatkowski warns AI will wipe out swathes of knowledge-work jobs across sectors like banking and translation; Klarna cut staff from 7,400 to about 3,000 while raising pay and plans to expand banking using its UK electronic-money licence.","site_name":"Bloomberg","url":"https:\/\/mercury.bloomberg.com\/news\/T3UYZQGOT0JK"},{"rating":8.8273896812,"summary":"British Standards Institute report finds 39% of business leaders have cut entry-level roles for AI, 43% plan further cuts, and 95% report no AI ROI, yet 55% deem workforce disruptions acceptable, risking weakened future leadership pipelines without upskilling.","site_name":"The Register","url":"https:\/\/www.theregister.com\/2025\/10\/10\/ai_is_displacing_entrylevel_professionals\/"},{"rating":8.5453745485,"summary":"Translators face significant job displacement from advancing AI language technologies,

10:16:13 | NewsletterAgent.newsletter_agent | INFO | User message: STORIES:
[{"rating":8.7548483797,"summary":"Google has integrated Gemini into Chrome, while Perplexity's Comet and Opera's Neon launched AI-native browsers that perform tasks like booking travel and managing email; privacy concerns over data access and inference have prompted firms to stress encryption and responsible practices.","site_name":"Fortune","url":"https:\/\/fortune.com\/2025\/10\/11\/ai-browser-wars-agents-google-perplexity-opera-comet-neon-gemini\/"},{"rating":8.2264829357,"summary":"AMD and Sony's Project Amethyst is a co-engineered PlayStation chipset design that rethinks the graphics pipeline using machine learning, new compression and neural arrays to share compute data so shaders process larger screen regions, improving rendering efficiency and upscaling.","site_name":"Ars Technica","url":"https:\/\/arstechnica.com\/gaming\/2025\/10\/amd-and-sony-tease-new-chip-architecture-ahead-of-playstation-6\/"},{"

10:16:13 | NewsletterAgent.newsletter_agent | INFO | User message: STORIES:
[{"rating":8.0229878524,"summary":"Nvidia developed a reinforcement-learning pretraining (RLP) method that trains LLMs to generate reasoning chains before token prediction, boosting reasoning-benchmark performance by up to 17% and improving downstream fine-tuning reliability for enterprise workflows.","site_name":"VentureBeat","url":"https:\/\/venturebeat.com\/ai\/nvidia-researchers-boost-llms-reasoning-skills-by-getting-them-to-think"},{"rating":7.0516591299,"summary":"Google DeepMind introduced Gemini Robotics 1.5 and Gemini Robotics-ER 1.5, models that enable robots such as Aloha 2 to perform complex multistep tasks with advanced perception, spatial reasoning and natural-language explanations, and to use tools like Google Search.","site_name":"Live Science","url":"https:\/\/www.livescience.com\/technology\/robotics\/robots-receive-major-intelligence-boost-thanks-to-google-deepminds-thinking-ai-a-pair-of-mode

10:16:14 | NewsletterAgent.newsletter_agent | INFO | User message: STORIES:
[{"rating":7.3723953334,"summary":"Apple is reportedly close to an acqui-hire of startup Prompt AI to acquire its talent and Seemour computer-vision technology for Apple Intelligence, with Prompt AI\u2019s Seemour app to be retired as the company winds down that business.","site_name":"Apple Insider","url":"https:\/\/appleinsider.com\/articles\/25\/10\/11\/ai-acqui-hire-could-boost-apple-intelligences-vision-capabilities"},{"rating":6.4453312441,"summary":"TCS has doubled its AI-skilled workforce to 160,000 and hired 18,500 in Q2, and plans to invest $5\u20136 billion to build one gigawatt of AI data-center capacity, starting with a 150 MW, ~$1 billion phase due in 14\u201318 months.","site_name":"The Economic Times","url":"https:\/\/m.economictimes.com\/markets\/expert-view\/tcs-ai-skilled-workforce-doubles-to-1-6-lakh-18500-new-hires-in-q2-with-focus-on-future-ready-skills\/articleshow\/124453937.cms"},{"rati

10:16:14 | NewsletterAgent.newsletter_agent | INFO | User message: STORIES:
[{"rating":6.6831775951,"summary":"Former OpenAI product manager Angela Jiang launched Worktrace AI to automate repetitive enterprise tasks by observing employees; investors include Mira Murati, OpenAI's startup fund, 8VC and Conviction, and it is raising a $10 million seed at a $50 million valuation.","site_name":"Business Insider","url":"https:\/\/www.businessinsider.com\/former-openai-product-manager-launches-worktrace-ai-mira-murati-backing-2025-10"},{"rating":6.584617738,"summary":"Govini says it surpassed $100 million in annual recurring revenue and secured a $150 million investment from Bain Capital to expand its AI Ark product and team for military supply-chain and product-lifecycle modernization.","site_name":"CNBC","url":"https:\/\/www.cnbc.com\/2025\/10\/10\/defense-tech-govini-palantir-revenue.html"},{"rating":5.6232630786,"summary":"ARC Group International provided ScanTech AI Systems an equity lin

10:16:26 | NewsletterAgent.newsletter_agent | INFO | Result: ## Deepfakes Go Mainstream

- OpenAI's Sora drives hyper-realistic deepfakes as platforms add safeguards; experts warn of rising misinformation, scams, and eroding trust - [NPR](https://www.npr.org/2025/10/10/nx-s1-5567162/sora-ai-openai-deepfake)
10:16:34 | NewsletterAgent.newsletter_agent | INFO | Result: ## AI Reasoning Leaps Ahead

- Nvidia's reinforcement-learning pretraining trains LLMs to think before predicting, boosting reasoning benchmarks up to 17% and improving enterprise fine-tuning reliability - [VentureBeat](https://venturebeat.com/ai/nvidia-researchers-boost-llms-reasoning-skills-by-getting-them-to-think)
- DeepMind unveils Gemini Robotics 1.5 and ER 1.5, enabling robots like Aloha 2 to do multistep tasks, spatial reasoning, natural-language explanations, and tool use - [Live Science](https://www.livescience.com/technology/robotics/robots-receive-major-intelligence-boost-thanks-to-google-deepminds-thinking-ai-

10:16:54 | NewsletterAgent.newsletter_agent | INFO | Result: ## AI Hiring Frenzy

- TCS doubles AI talent to 160,000, commits $5–6B for 1GW AI data centers, and hires 18,500 in Q2 - [The Economic Times](https://m.economictimes.com/markets/expert-view/tcs-ai-skilled-workforce-doubles-to-1-6-lakh-18500-new-hires-in-q2-with-focus-on-future-ready-skills/articleshow/124453937.cms)
- TCS opens London AI Experience Zone, targets 5,000 UK jobs in three years - [The Hindu Business Line](https://www.thehindubusinessline.com/info-tech/tcs-launches-ai-hub-design-studio-in-london-to-create-5000-uk-jobs-in-3-years/article70147361.ece)
- Apple nears Prompt AI acqui-hire to boost Apple Intelligence vision; Seemour app to be retired - [Apple Insider](https://appleinsider.com/articles/25/10/11/ai-acqui-hire-could-boost-apple-intelligences-vision-capabilities)
- Campus hires ex-Meta AI chief Jerome Pesenti as CTO, acquires Sizzle AI to power its college learning platform - [Biztoc](https://biztoc.com/x/e

10:18:43 | NewsletterAgent.newsletter_agent | INFO | Result: ## AI Everywhere Now

- AI browser wars heat up: Gemini in Chrome, Perplexity Comet, Opera Neon push agentic web—privacy stakes rise - [Fortune](https://fortune.com/2025/10/11/ai-browser-wars-agents-google-perplexity-opera-comet-neon-gemini/)
- Users can block Google AI Overviews with a swearword, spotlighting rising reliance—and skepticism—around AI search - [The Guardian](https://www.theguardian.com/technology/2025/oct/11/using-a-swearword-in-your-google-search-can-stop-the-ai-answer-but-should-you)
- From prompts to purchases: AI agents become personal shoppers, promise frictionless commerce and new retail models - [Financial Times](https://www.ft.com/content/f312843b-1921-4998-963d-820885349f32) [Biztoc](https://biztoc.com/x/2f06edea358d9504)
- TCS makes AI default: 160,000 trained, $10B in AI-linked deals, 1 GW new data center capacity planned - [The Economic Times](https://m.economictimes.com/markets/expert-view/tcs-bet

In [67]:
draft_results[0]


('Workforce AI Impact',
 Section(section_title='Jobs at Stake, Skills in Demand', headlines=[SectionStory(headline='BSI: 39% cut entry-level jobs for AI; 43% plan more; 95% see no ROI', links=[SectionStoryLink(url='https://www.theregister.com/2025/10/10/ai_is_displacing_entrylevel_professionals/', site_name='The Register')], prune=False), SectionStory(headline='Klarna CEO warns AI will wipe out knowledge-work jobs; firm shrank staff 7,400 to ~3,000, eyes UK e-money banking expansion', links=[SectionStoryLink(url='https://mercury.bloomberg.com/news/T3UYZQGOT0JK', site_name='Bloomberg')], prune=False), SectionStory(headline='AI upends translation work; linguists report job losses and emotional strain as employers push automation', links=[SectionStoryLink(url='https://www.ft.com/content/50b1f03e-1d10-4a7c-afa3-b48e9a8d5133', site_name='Financial Times')], prune=False), SectionStory(headline='Google Cloud CEO: AI augments jobs; 10% engineer productivity gains; hiring ramps; support workloa

In [68]:
            sections_drafted = 0
            for result in draft_results:
                if isinstance(result, Exception):
                    self.logger.error(f"Error drafting section: {result}")
                    continue

                cat, content = result
                # state.newsletter_section_obj[cat] = content
                state.newsletter_section_text[cat] = content
                sections_drafted += 1

In [71]:
for k, v in state.newsletter_section_text.items():
    display(Markdown(str(v).replace("$", "\\\$")))


## Jobs at Stake, Skills in Demand

- BSI: 39% cut entry-level jobs for AI; 43% plan more; 95% see no ROI - [The Register](https://www.theregister.com/2025/10/10/ai_is_displacing_entrylevel_professionals/)
- Klarna CEO warns AI will wipe out knowledge-work jobs; firm shrank staff 7,400 to ~3,000, eyes UK e-money banking expansion - [Bloomberg](https://mercury.bloomberg.com/news/T3UYZQGOT0JK)
- AI upends translation work; linguists report job losses and emotional strain as employers push automation - [Financial Times](https://www.ft.com/content/50b1f03e-1d10-4a7c-afa3-b48e9a8d5133)
- Google Cloud CEO: AI augments jobs; 10% engineer productivity gains; hiring ramps; support workloads eased without layoffs - [Business Insider](https://www.businessinsider.com/google-cloud-ceo-ai-automate-jobs-2025-10)
- Sanders to propose robot tax on corporations replacing workers with AI, aiming to offset lost revenue and curb displacement - [Slashdot Linux](https://slashdot.org/submission/17341310/sanders-plans-ai-tax-legislation-to-save-jobs)
- Widening AI skills gap fuels labor shortages; nearly half get no training, mishandle data; leaders urged to prioritize upskilling and risk assessment - [TechTarget](https://www.techtarget.com/searchhrsoftware/feature/The-AI-skills-gap-is-getting-worse-Why-and-what-to-do)
- Microsoft pledges \\$4B to give free AI tools and training to all Washington schools, targeting 25M learners by 2025–2026 with Code.org - [Slashdot Linux](https://news.slashdot.org/story/25/10/11/0012220/microsoft-to-provide-free-ai-tools-for-washington-state-schools)

## OpenAI's Power Plays

- OpenAI anchors \\$1T web of tech deals, deepening industry reliance on its AI - [Financial Times](https://www.ft.com/content/4e39d081-ab26-4bc2-9c4c-256d766f28e2)
- OpenAI buys 10% of AMD in multibillion-dollar deal; AMD shares jump 34% - [Marketplace](https://www.marketplace.org/episode/2025/10/10/openai-partners-with-amd-google-home-devices-get-ai-update)
- OpenAI valued at \\$500B, stays private; investors seek exposure via Microsoft, Nvidia, or DXYZ - [247wallst](https://247wallst.com/investing/2025/10/11/invest-here-to-get-a-piece-of-openai/)
- Duolingo reportedly top OpenAI customer by token use; 34.2M DAUs, 9.5M subscribers in 2024 - [Wccftech](https://wccftech.com/duolingo-allegedly-tops-a-list-of-openais-top-30-customers-by-token-consumption/)
- Mark Cuban okays Sora 2 likeness videos, each promoting Cost Plus Drugs as they go viral - [Business Insider](https://www.businessinsider.com/mark-cuban-free-advertisements-for-cost-plus-drugs-sora-2-2025-10)

## GPT-5 Bias Check

- OpenAI says GPT-5 cuts political bias 30% after 100-topic stress test; new tone controls and guidelines debut as outside studies still flag broader AI biases. - [The Verge](https://www.theverge.com/news/798388/openai-chatgpt-political-bias-eval) [Digital Trends](https://www.digitaltrends.com/computing/openai-says-chatgpt-is-the-least-biased-it-has-ever-been-but-its-not-all-roses/)

## OpenAI Under Legal Scrutiny

- OpenAI subpoenas AI-regulation advocates in Musk countersuit, seeking messages and alleged Musk funding tied to Encode challenge - [The Verge](https://www.theverge.com/news/798523/openai-ai-regulation-advocates-subpoenas-police)
- Court lifts ChatGPT deletion-log preservation order; OpenAI can stop retaining most deleted/temporary chats from Sept. 26, with some users still monitored - [Ars Technica](https://arstechnica.com/tech-policy/2025/10/openai-no-longer-forced-to-save-deleted-chats-but-some-users-still-affected/)
- Column: Altman’s Sora 2 faces copyright backlash as OpenAI’s Nvidia and AMD partnerships help drive over \\$1 trillion in deals this year - [The Guardian](https://www.theguardian.com/commentisfree/2025/oct/10/sam-altman-copyright-ai-sora-2-video-generator)

## OpenAI: Speed and Scale

- Altman's OpenAI races ahead: over \\$13B revenue this year, but \\$155B compute costs through 2029 draw industry acclaim and consumer safety concerns - [Business Insider](https://www.businessinsider.com/tech-ceos-weigh-in-openai-chief-sam-altman-leadership-2025-10)
- OpenAI's Sora app debuts: text-to-video creation, customizable Cameos avatars, editing, remixing, social sharing, privacy controls, and watermarked downloads - [Geeky Gadgets](https://www.geeky-gadgets.com/sora-app-mobile-ai-video-creation/)

...

## AI Superpowers For Coding

- Anthropic unveils Claude Code 'Superpowers' plugin: autonomous planning, TDD, parallel git worktrees, skills, delegation; sharing and memory features coming. - [FSCK](https://blog.fsck.com/2025/10/09/superpowers/)
- Live demo: Developer uses Claude Code with crowd prompts and voice to prototype a basket‑weaving cursor via fuzzy spec compilation at Denver meetup. - [Hacker Noon](https://hackernoon.com/can-you-improvise-software-one-developer-put-ai-to-the-test)
- Bitrig for iOS turns voice or prompts into personalized apps for non‑coders, but many generated tools suffer from interaction bugs like unresponsive buttons. - [Tom's Guide](https://www.tomsguide.com/ai/i-tried-an-app-that-makes-apps-using-ai-on-my-iphone-and-it-felt-like-the-future)

## AI Money Moves

- Malaysia commits RM5.9b cross-ministry to accelerate AI by 2030, funds National AI Office, MSME tax breaks, RM2b Sovereign AI Cloud. - [The Star](https://www.thestar.com.my/tech/tech-news/2025/10/10/anwar-rm59bil-cross-ministry-allocation-to-keep-m039sia-at-forefront-of-ai-development)
- EU unveils six AI factories across Czech Republic, Lithuania, Netherlands, Romania, Spain and Poland to boost manufacturing capacity and deployment. - [Biztoc](https://biztoc.com/x/04127493ca266921)
- Nvidia seeks alternative sites for Taiwan headquarters as Beitou-Shilin project stalls over surface rights; regulators weigh fixes to keep investment. - [The Economic Times](https://m.economictimes.com/tech/tech-bytes/nvidia-seeks-alternative-sites-for-its-new-taiwan-headquarters/articleshow/124453187.cms)
- U.S. AI diagnostics market to grow from \\$424.41M in 2024 to \\$1.83B by 2033, 17.68% CAGR, driven by precision medicine and imaging analytics. - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/10/3164756/28124/en/United-States-Artificial-Intelligence-in-Diagnostics-Market-Research-Report-2025-2033-Precision-Medicine-Advanced-Imaging-Analytics-and-AI-Enabled-Clinical-Decision-Support-Spur-Ex.html)
- Oracle's AI alliances lift OCI revenue 53% and backlog to \\$455B; Palantir offers faster growth with 128% net-dollar retention but higher valuation risk. - [Forbes](https://www.forbes.com/sites/investor-hub/article/is-oracle-or-palantir-the-smarter-ai-stock-to-buy/)
- Nvidia dominates AI chips with \\$26.4B net income and \\$56.8B cash; CoreWeave revenue quadruples but remains unprofitable. - [The Motley Fool](https://www.fool.com/investing/2025/10/10/better-ai-stock-to-buy-coreweave-or-nvidia/)
- Citadel trims Broadcom over 80%, boosts Nvidia to 8M+ shares as Nvidia rebounds 90% in six months. - [The Motley Fool](https://www.fool.com/investing/2025/10/11/billionaire-ken-griffin-citadel-broadcom-stock-ai/)
- Berkshire Hathaway concentrates 24% of \\$304B portfolio in AI-linked Apple and Amazon, with Apple topping \\$70B. - [The Motley Fool](https://www.fool.com/investing/2025/10/10/warren-buffett-ai-stocks-berkshire-hathaway/)
- Three AI ETFs to watch: AIQ, BAI, and ROBO; fees from 0.55% to 0.95%, AIQ up 30% YTD. - [The Motley Fool](https://www.fool.com/investing/2025/10/10/3-ai-etfs-to-buy-now-for-the-coming-tech-revolutio/)

## AI Funding Roundup

- Govini surpasses \\$100M ARR, secures \\$150M from Bain to scale AI Ark for defense supply-chain and lifecycle modernization - [CNBC](https://www.cnbc.com/2025/10/10/defense-tech-govini-palantir-revenue.html)
- Prezent raises \\$30M at \\$400M valuation to acquire AI services firms, starting with Prezentium; total funding tops \\$74M - [TechCrunch](https://techcrunch.com/2025/10/10/prezent-raises-30-million-to-acquire-ai-services-firms-starting-with-founders-other-company/)
- Ex-OpenAI PM launches Worktrace AI, raising \\$10M seed at \\$50M valuation with backing from Mira Murati, OpenAI Startup Fund, 8VC, Conviction - [Business Insider](https://www.businessinsider.com/former-openai-product-manager-launches-worktrace-ai-mira-murati-backing-2025-10)
- ARC Group provides ScanTech AI up to \\$50M equity line, including \\$500K PIPE, to fund SENTINEL platform, Nasdaq compliance, and 2026 certification - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/10/3164812/0/en/ARC-Group-Invests-in-ScanTech-AI-Systems-Backing-Company-s-AI-Powered-Security-and-Regulatory-Alignment.html)

## AI Reckoning: Hype vs. Harm

- Dimon, BoE, Bezos warn AI bubble could burst within 6–24 months, with Europe exposed - [RT\u00c9](https:\/\/www.rte.ie\/news\/business\/2025\/1010\/1537856-ai-stock-tech\/)
- Ex-Twitter\/Meta policy chief: AI repeats social media mistakes; urgent need for testing, regulation, global standards - [Fortune](https:\/\/fortune.com\/2025\/10\/11\/front-row-seat-social-media-revolution-same-mistakes-ai\/)
- 82% of CEOs call AI essential, but only 41% align on product vision; experts warn copying features undermines strategy - [Forbes](https:\/\/www.forbes.com\/sites\/annegriffin\/2025\/10\/10\/when-ai-undermines-product-strategy-and-what-to-do-instead\/)
- Inside Google’s AI juggling act: innovation vs. safety, regulation, and revenue pressures - [The Wall Street Journal](https:\/\/www.wsj.com\/tech\/ai\/google-search-ai-business-48e13735)
- How to curb AI in Google apps: you can disable many features, but not all - [CNET](https:\/\/www.cnet.com\/tech\/services-and-software\/how-to-turn-off-or-reduce-ai-in-google-apps-and-services\/)
- Beware AI meeting bots: auto-joining notetakers trigger bans; consider non-joining or native transcription - [Reddit](https:\/\/www.reddit.com\/r\/ChatGPT\/comments\/1o3i2zw\/do_not_use_ai_notetakers_that_join_your_calls\/)
- Perplexity’s Comet can ace online courses in seconds; CEO warns against cheating and prompt-injection risks - [Benzinga](https:\/\/www.benzinga.com\/news\/education\/25\/10\/48164089\/absolutely-dont-do-this-perplexity-ceo-aravind-srinivas-warns-against-misuse-of-ai-tools)
- AI ‘homeless man’ TikTok prank sparks false 911 calls in U.S. and U.K., wasting emergency resources - [Gizmodo](https:\/\/gizmodo.com\/police-say-people-keep-calling-911-over-an-ai-homeless-man-tiktok-prank-2000671130)
- Ex–UK PM Rishi Sunak takes advisory roles at Microsoft and Anthropic, pledges pay to charity, stirs ethics debate - [International Business Times](https:\/\/www.ibtimes.co.uk\/ex-pm-rishi-sunak-lands-shocking-new-job-microsoft-ai-giant-anthropic-1747272)
- Parents weigh AI for kids: creative tools vs. privacy, safety, and ethics at ‘Raising Kids in the Age of AI’ - [Business Insider](https:\/\/www.businessinsider.com\/ai-workshop-parenting-new-generation-kids-technology-chatgpt-gemini-exposure-2025-10)

## AI Supercharges Research Search

- AI research tool roundup: Scite, Semantic Scholar, Elicit, Trinka streamline literature search, writing, and data analysis. - [Geeky Gadgets](https://www.geeky-gadgets.com/ai-research-tools-2025/)
- ThalamusDB brings NLfilter/NLjoin to SQL for multimodal search on DuckDB, streaming approximate answers with error bounds via LLMs. - [GitHub Gist](https://github.com/itrummer/thalamusdb)
- Mixedbread Search enables natural-language queries across 50,000+ National Gallery of Art images for intuitive discovery. - [Mixed Bread](https://nga.demo.mixedbread.com/)

In [69]:
cat_df = state.headline_df.groupby("cat") \
    .count() \
    .reset_index()[['cat','source']] \
    .sort_values('source', ascending=False)
output_str = ""
for cat in cat_df["cat"]:
    if cat != "Other":
        output_str += str(state.newsletter_section_text[cat]) + "\n\n"
        display(Markdown(str(state.newsletter_section_text[cat]).replace("$", "\\\$")))

# first do full rewrite .
# check vs. objects , not showing the ones market for pruning
# move prune=True to Other

## AI Everywhere Now

- AI browser wars heat up: Gemini in Chrome, Perplexity Comet, Opera Neon push agentic web—privacy stakes rise - [Fortune](https://fortune.com/2025/10/11/ai-browser-wars-agents-google-perplexity-opera-comet-neon-gemini/)
- Users can block Google AI Overviews with a swearword, spotlighting rising reliance—and skepticism—around AI search - [The Guardian](https://www.theguardian.com/technology/2025/oct/11/using-a-swearword-in-your-google-search-can-stop-the-ai-answer-but-should-you)
- From prompts to purchases: AI agents become personal shoppers, promise frictionless commerce and new retail models - [Financial Times](https://www.ft.com/content/f312843b-1921-4998-963d-820885349f32) [Biztoc](https://biztoc.com/x/2f06edea358d9504)
- TCS makes AI default: 160,000 trained, \\$10B in AI-linked deals, 1 GW new data center capacity planned - [The Economic Times](https://m.economictimes.com/markets/expert-view/tcs-bets-big-on-ai-and-data-centres-every-project-will-be-ai-led-says-ceo-k-krithivasan/articleshow/124451159.cms)
- TCS buys Salesforce partner ListEngage for up to \\$72.8M, bolstering marketing cloud capabilities; Q2 revenue \\$7.46B - [Verdict](https://www.verdict.co.uk/news/tcs-buys-listengage-to-expand-salesforce/)
- Gap signs multi-year Google Cloud AI alliance to speed design, pricing and hyper-personalized marketing; Q2 FY2025 net sales \\$3.7B - [Retail Insight Network](https://www.retail-insight-network.com/news/gap-collaborates-google-cloud-ai/)
- Barclays backs Manhattan Associates and Trimble as top agentic-AI industrial bets amid rising recurring revenue and margin gains - [Yahoo](https://finance.yahoo.com/news/barclays-says-2-industrial-tech-100352625.html)
- Chevron opens 312,000-square-foot Bengaluru ENGINE hub to scale digital and AI, plans ~\\$1B India investment amid \\$3B cost-cut push - [Yahoo](https://finance.yahoo.com/news/chevron-expands-india-hub-boost-115647358.html)
- Middle East AI adoption surges: ~60% report fast uptake, but only 14–28% at scale; GCC CEOs near 90% GenAI use, cyber risks >60% - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/10/3164741/28124/en/AI-in-the-Middle-East-Adoption-Trends-Readiness-and-Risk-Landscape-Analysis-Report-2025.html)
- Experts urge accelerating safe AI in hospitals to unlock better diagnostics, personalized care and efficiency—requiring investment, policy and collaboration - [Financial Times](https://www.ft.com/content/83f18513-137e-4b9c-8c7b-b0b45e0d7e39)
- US campaigns ramp up AI for fundraising, targeting and mobilization, raising regulation, misuse and misinformation concerns ahead of midterms - [The American Prospect](https://prospect.org/power/2025-10-10-ai-artificial-intelligence-campaigns-midterms/)
- Creator economy to hit \\$600B by 2030, driven by AI workflows and hyper-specialization, says ThriveCart; platform processes \\$2B+ annually - [Financial Post](https://financialpost.com/globe-newswire/creator-economy-to-reach-600b-by-2030-thrivecart-creator-report-highlights-ai-and-specialization-will-fuel-growth)
- AMD and Sony's Project Amethyst reimagines PlayStation graphics with ML-driven compression and neural arrays to boost rendering efficiency and upscaling - [Ars Technica](https://arstechnica.com/gaming/2025/10/amd-and-sony-tease-new-chip-architecture-ahead-of-playstation-6/)
- AI tools can personalize learning and improve accessibility, but a new divide is emerging between effective users and those left behind - [Forbes](https://www.forbes.com/sites/danfitzpatrick/2025/10/10/can-ai-help-make-education-more-inclusive/)
- Autonomous passenger and cargo aircraft near reality, promising new services and changes to pilot roles, operations and airspace management - [The Wall Street Journal](https://www.wsj.com/tech/ai/these-planes-that-fly-themselves-could-transform-the-skies-d5d1b7a1)
- Snaptrude turns text into editable 3D building models, integrating live code compliance, cost and standards data for faster design planning - [Designboom](https://www.designboom.com/architecture/snaptrude-ai-generate-editable-3d-architecture-models-simple-text-descriptions-archdaily-10-10-2025/)
- Caira turns iPhone into mirrorless camera with larger sensor, MFT lenses and on-device AI editing; Kickstarter launches Oct. 30 - [Creative Bloq](https://www.creativebloq.com/ai/ai-art/could-this-iphone-nano-banana-camera-finally-make-ai-photography-a-thing)

## AI Reckoning: Hype vs. Harm

- Dimon, BoE, Bezos warn AI bubble could burst within 6–24 months, with Europe exposed - [RT\u00c9](https:\/\/www.rte.ie\/news\/business\/2025\/1010\/1537856-ai-stock-tech\/)
- Ex-Twitter\/Meta policy chief: AI repeats social media mistakes; urgent need for testing, regulation, global standards - [Fortune](https:\/\/fortune.com\/2025\/10\/11\/front-row-seat-social-media-revolution-same-mistakes-ai\/)
- 82% of CEOs call AI essential, but only 41% align on product vision; experts warn copying features undermines strategy - [Forbes](https:\/\/www.forbes.com\/sites\/annegriffin\/2025\/10\/10\/when-ai-undermines-product-strategy-and-what-to-do-instead\/)
- Inside Google’s AI juggling act: innovation vs. safety, regulation, and revenue pressures - [The Wall Street Journal](https:\/\/www.wsj.com\/tech\/ai\/google-search-ai-business-48e13735)
- How to curb AI in Google apps: you can disable many features, but not all - [CNET](https:\/\/www.cnet.com\/tech\/services-and-software\/how-to-turn-off-or-reduce-ai-in-google-apps-and-services\/)
- Beware AI meeting bots: auto-joining notetakers trigger bans; consider non-joining or native transcription - [Reddit](https:\/\/www.reddit.com\/r\/ChatGPT\/comments\/1o3i2zw\/do_not_use_ai_notetakers_that_join_your_calls\/)
- Perplexity’s Comet can ace online courses in seconds; CEO warns against cheating and prompt-injection risks - [Benzinga](https:\/\/www.benzinga.com\/news\/education\/25\/10\/48164089\/absolutely-dont-do-this-perplexity-ceo-aravind-srinivas-warns-against-misuse-of-ai-tools)
- AI ‘homeless man’ TikTok prank sparks false 911 calls in U.S. and U.K., wasting emergency resources - [Gizmodo](https:\/\/gizmodo.com\/police-say-people-keep-calling-911-over-an-ai-homeless-man-tiktok-prank-2000671130)
- Ex–UK PM Rishi Sunak takes advisory roles at Microsoft and Anthropic, pledges pay to charity, stirs ethics debate - [International Business Times](https:\/\/www.ibtimes.co.uk\/ex-pm-rishi-sunak-lands-shocking-new-job-microsoft-ai-giant-anthropic-1747272)
- Parents weigh AI for kids: creative tools vs. privacy, safety, and ethics at ‘Raising Kids in the Age of AI’ - [Business Insider](https:\/\/www.businessinsider.com\/ai-workshop-parenting-new-generation-kids-technology-chatgpt-gemini-exposure-2025-10)

## AI Money Moves

- Malaysia commits RM5.9b cross-ministry to accelerate AI by 2030, funds National AI Office, MSME tax breaks, RM2b Sovereign AI Cloud. - [The Star](https://www.thestar.com.my/tech/tech-news/2025/10/10/anwar-rm59bil-cross-ministry-allocation-to-keep-m039sia-at-forefront-of-ai-development)
- EU unveils six AI factories across Czech Republic, Lithuania, Netherlands, Romania, Spain and Poland to boost manufacturing capacity and deployment. - [Biztoc](https://biztoc.com/x/04127493ca266921)
- Nvidia seeks alternative sites for Taiwan headquarters as Beitou-Shilin project stalls over surface rights; regulators weigh fixes to keep investment. - [The Economic Times](https://m.economictimes.com/tech/tech-bytes/nvidia-seeks-alternative-sites-for-its-new-taiwan-headquarters/articleshow/124453187.cms)
- U.S. AI diagnostics market to grow from \\$424.41M in 2024 to \\$1.83B by 2033, 17.68% CAGR, driven by precision medicine and imaging analytics. - [Globe Newswire](https://www.globenewswire.com/news-release/2025/10/10/3164756/28124/en/United-States-Artificial-Intelligence-in-Diagnostics-Market-Research-Report-2025-2033-Precision-Medicine-Advanced-Imaging-Analytics-and-AI-Enabled-Clinical-Decision-Support-Spur-Ex.html)
- Oracle's AI alliances lift OCI revenue 53% and backlog to \\$455B; Palantir offers faster growth with 128% net-dollar retention but higher valuation risk. - [Forbes](https://www.forbes.com/sites/investor-hub/article/is-oracle-or-palantir-the-smarter-ai-stock-to-buy/)
- Nvidia dominates AI chips with \\$26.4B net income and \\$56.8B cash; CoreWeave revenue quadruples but remains unprofitable. - [The Motley Fool](https://www.fool.com/investing/2025/10/10/better-ai-stock-to-buy-coreweave-or-nvidia/)
- Citadel trims Broadcom over 80%, boosts Nvidia to 8M+ shares as Nvidia rebounds 90% in six months. - [The Motley Fool](https://www.fool.com/investing/2025/10/11/billionaire-ken-griffin-citadel-broadcom-stock-ai/)
- Berkshire Hathaway concentrates 24% of \\$304B portfolio in AI-linked Apple and Amazon, with Apple topping \\$70B. - [The Motley Fool](https://www.fool.com/investing/2025/10/10/warren-buffett-ai-stocks-berkshire-hathaway/)
- Three AI ETFs to watch: AIQ, BAI, and ROBO; fees from 0.55% to 0.95%, AIQ up 30% YTD. - [The Motley Fool](https://www.fool.com/investing/2025/10/10/3-ai-etfs-to-buy-now-for-the-coming-tech-revolutio/)

## Building the AI Backbone

- Google commits \\$10B to 1 GW data center cluster near Visakhapatnam, full operations by July 2028 - [Entrackr](https://entrackr.com/snippets/google-to-invest-10-bn-in-visakhapatnam-data-centre-cluster-10540536)
- EU adds six AI factory sites to expand supercomputing access across Czechia, Lithuania, Netherlands, Romania, Spain and Poland - [DPA International](https://dpa-international.com/economics/urn:newsml:dpa.com:20090101:251010-99-291550/)
- India deploys 38,000 GPUs; two foundational models by end-2025; sovereign AI model targeted for February 2026 - [The Economic Times](https://m.economictimes.com/tech/technology/indias-sovereign-ai-model-to-be-ready-by-february-meity-secretary/articleshow/124452012.cms)
- CoreWeave touts \\$50B+ backlog, \\$14.2B Meta and \\$22.4B OpenAI deals; plans \\$20–23B 2025 capex, sees 2.8x revenue growth - [The Motley Fool](https://www.fool.com/investing/2025/10/10/prediction-ai-infrastructure-stock-market-leader/)
- Applied Digital Q1 revenue up 84% to \\$64.2M, topping estimates as AI demand and 150MW CoreWeave lease fuel data center expansion - [Yahoo](https://finance.yahoo.com/news/applied-digital-beats-revenue-estimates-212255955.html)
- Solidigm opens AI Central Lab with 23.6 PB in 16U, 116 GB/s per-node storage; trials cut DRAM use up to 57% - [TechPowerUp](https://www.techpowerup.com/341765/solidigm-ai-central-lab-features-192-ssds-packing-23-6-pb-into-16u)
- Google says it processes 1.3 quadrillion AI tokens monthly; critics question metric inflation and environmental accounting - [The Decoder](https://the-decoder.com/google-boasts-1-3-quadrillion-tokens-each-month-but-the-figure-is-mostly-window-dressing/)

## Jobs at Stake, Skills in Demand

- BSI: 39% cut entry-level jobs for AI; 43% plan more; 95% see no ROI - [The Register](https://www.theregister.com/2025/10/10/ai_is_displacing_entrylevel_professionals/)
- Klarna CEO warns AI will wipe out knowledge-work jobs; firm shrank staff 7,400 to ~3,000, eyes UK e-money banking expansion - [Bloomberg](https://mercury.bloomberg.com/news/T3UYZQGOT0JK)
- AI upends translation work; linguists report job losses and emotional strain as employers push automation - [Financial Times](https://www.ft.com/content/50b1f03e-1d10-4a7c-afa3-b48e9a8d5133)
- Google Cloud CEO: AI augments jobs; 10% engineer productivity gains; hiring ramps; support workloads eased without layoffs - [Business Insider](https://www.businessinsider.com/google-cloud-ceo-ai-automate-jobs-2025-10)
- Sanders to propose robot tax on corporations replacing workers with AI, aiming to offset lost revenue and curb displacement - [Slashdot Linux](https://slashdot.org/submission/17341310/sanders-plans-ai-tax-legislation-to-save-jobs)
- Widening AI skills gap fuels labor shortages; nearly half get no training, mishandle data; leaders urged to prioritize upskilling and risk assessment - [TechTarget](https://www.techtarget.com/searchhrsoftware/feature/The-AI-skills-gap-is-getting-worse-Why-and-what-to-do)
- Microsoft pledges \\$4B to give free AI tools and training to all Washington schools, targeting 25M learners by 2025–2026 with Code.org - [Slashdot Linux](https://news.slashdot.org/story/25/10/11/0012220/microsoft-to-provide-free-ai-tools-for-washington-state-schools)

...

## GPT-5 Bias Check

- OpenAI says GPT-5 cuts political bias 30% after 100-topic stress test; new tone controls and guidelines debut as outside studies still flag broader AI biases. - [The Verge](https://www.theverge.com/news/798388/openai-chatgpt-political-bias-eval) [Digital Trends](https://www.digitaltrends.com/computing/openai-says-chatgpt-is-the-least-biased-it-has-ever-been-but-its-not-all-roses/)

## AI Performance, Measured

- InferenceMAX: Open-source LLM inference benchmark spans NVIDIA, AMD, TPU, Trainium; tracks throughput, latency, cost per token on LLaMA 70B, DeepSeek 670B. - [SemiAnalysis](https://newsletter.semianalysis.com/p/inferencemax-open-source-inference)
- Raindrop's Experiments brings A/B testing to AI agents; measures model, prompt, tool changes with feature-flag integration, root-cause metrics, PII redaction, at \\$350/month. - [VentureBeat](https://venturebeat.com/ai/will-updating-your-ai-agents-help-or-hamper-their-performance-raindrops-new)

## Deepfakes Go Mainstream

- OpenAI's Sora drives hyper-realistic deepfakes as platforms add safeguards; experts warn of rising misinformation, scams, and eroding trust - [NPR](https://www.npr.org/2025/10/10/nx-s1-5567162/sora-ai-openai-deepfake)

## OpenAI: Speed and Scale

- Altman's OpenAI races ahead: over \\$13B revenue this year, but \\$155B compute costs through 2029 draw industry acclaim and consumer safety concerns - [Business Insider](https://www.businessinsider.com/tech-ceos-weigh-in-openai-chief-sam-altman-leadership-2025-10)
- OpenAI's Sora app debuts: text-to-video creation, customizable Cameos avatars, editing, remixing, social sharing, privacy controls, and watermarked downloads - [Geeky Gadgets](https://www.geeky-gadgets.com/sora-app-mobile-ai-video-creation/)

## AI Reasoning Leaps Ahead

- Nvidia's reinforcement-learning pretraining trains LLMs to think before predicting, boosting reasoning benchmarks up to 17% and improving enterprise fine-tuning reliability - [VentureBeat](https://venturebeat.com/ai/nvidia-researchers-boost-llms-reasoning-skills-by-getting-them-to-think)
- DeepMind unveils Gemini Robotics 1.5 and ER 1.5, enabling robots like Aloha 2 to do multistep tasks, spatial reasoning, natural-language explanations, and tool use - [Live Science](https://www.livescience.com/technology/robotics/robots-receive-major-intelligence-boost-thanks-to-google-deepminds-thinking-ai-a-pair-of-models-that-help-machines-understand-the-world)

In [ ]:
print(output_str)


In [ ]:
draft_newsletter_system_prompt, draft_newsletter_user_prompt, model = \
    LangfuseClient().get_probmpt("newsagent/draft_newsletter")


In [ ]:
class Mystr(BaseModel):
    """A string"""
    mystr: str = Field(
        description="a string")


In [ ]:
draft_newsletter_agent = LLMagent(
    system_prompt=draft_newsletter_system_prompt,
    user_prompt=draft_newsletter_user_prompt,
    output_type=Mystr,
    model=model,
    verbose=True,
    logger=logger
)


In [ ]:
# Apply prompt to generate final newsletter
newsletter_content = await draft_newsletter_agent.run_prompt(input_str=output_str)


In [ ]:
newsletter_content = newsletter_content.mystr
display(Markdown(newsletter_content))


In [ ]:
HTML(newsletter_content)

In [ ]:
%pip install markdown 
from utilities import send_gmail
import markdown

In [ ]:
newsletter_content_html = markdown.markdown(newsletter_content)

In [ ]:
                today = datetime.now().strftime("%B %d, %Y")
                subject = f"AI News Digest - {today}"

                # Apply HTML styling
                html_content = f"""
                <div style="max-width: 800px; margin: 0 auto; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial, sans-serif;">
                    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 40px 20px; text-align: center; border-radius: 8px 8px 0 0;">
                        <h1 style="color: white; margin: 0; font-size: 32px;">AI News Digest</h1>
                        a<p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">{today}</p>
                    </div>
                    <div style="background: #ffffff; padding: 30px; border-radius: 0 0 8px 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                        {newsletter_content_html}
                    </div>
                    <div style="text-align: center; padding: 20px; color: #666; font-size: 14px;">
                        <p>Generated on {today} by AI Newsletter Agent</p>
                    </div>
                </div>
                """

                send_gmail(subject, html_content)

In [ ]:
use mmr 
do the pruning 
rewrite other after pruning
need to pass thinking effort
critic optimizer loop

In [ ]:
input_text = headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")
input_text


In [ ]:
write_section_system_prompt, write_section_user_prompt, model = \
    LangfuseClient().get_prompt("newsagent/write_section")

write_section_agent = LLMagent(
    system_prompt=write_section_system_prompt,
    user_prompt=write_section_user_prompt,
    output_type=Section,
    model=model,
    verbose=True,
    logger=logger
)


In [ ]:
response = await write_section_agent.run_prompt(input_text=input_text)
response

In [ ]:
display(Markdown(str(response)))

In [ ]:
class Link(BaseModel):
    url: str = Field(description="URL of the article")
    site_name: str = Field(description="Name of the website/source")
    def __str__(self):
        return f"[{self.site_name}]({self.url})"


class Story(BaseModel):
    headline: str = Field(description="Summary of the story")
    links: List[Link] = Field(description="List of links related to this story")
    prune: bool = Field(description="Whether to prune/exclude this story")
    def __str__(self):
        return "" if self.prune else f"- {self.headline} - " + " ".join([str(s) for s in self.links])


class Section(BaseModel):
    section_title: str = Field(description="Title of the newsletter section")
    headlines: List[Story] = Field(description="List of stories in this section")
    def __str__(self):
        return f"## {self.section_title}\n\n" + "\n".join(
            [str(h) for h in self.headlines]
        )
        

In [ ]:
headline_df=state.headline_df
headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")


In [ ]:
print("# SUGGESTED TOPICS:")
catcount = headline_df.groupby("cat").count().reset_index()[['cat', 'source']].sort_values('source', ascending=False)
for c in catcount["cat"]:
    print(c)
print()
print("# RAW NEWS ITEMS:")
i =0
for row in headline_df.sort_values(["cat", "rating"], ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"Rating: {row.rating:.1f}\n")    
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")
    i +=1


In [ ]:
headline_df['cat'].unique() 


In [ ]:
astate.get_completed_steps() 



In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"{row.rating:.1f}"))   
    display(Markdown(f"[{row.title}]({row.url}) - {row.site_name}".replace("$","\\\\$")))
    row_topics = ", ".join(row.topics)
    display(Markdown(f"Topics: {row_topics}".replace("$","\\\\$")))
    display(Markdown(f"{row.short_summary}".replace("$","\\\\$")))
    display(Markdown(f"{row.summary}".replace("$","\\\\$")))
    i +=1
#     if i>=30:
#         break

In [ ]:
class DistilledStory(BaseModel):
    """DistilledStory class for structured output distillation into a single sentence """
    item: str = Field(description="List of StoryRating")
        
system, user, model = LangfuseClient().get_prompt("newsagent/item_distiller")

distill_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=DistilledStory,
            model=model,
            verbose=False,
            logger=logger
        )

In [ ]:
response = await distill_agent.run_prompt(input_text="""AI 'Homeless Man' Challenge Sparks Outrage as Police Called Over Dangerous Viral Trend - International Business Times

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Rating: 1.5

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Parents in shock after dangerous 'homeless man' Snapchat AI prank goes horribly wrong, police called to calm the viral chaos.

A viral AI-driven prank called the 'homeless man' challenge on Snapchat deceived parents into thinking a homeless person had broken into their home, prompting police intervention.
The prank caused significant public backlash due to the misuse of AI technology to create real emergency scares, leading to police questioning the pranksters and debates about the legal consequences.
Experts emphasize the ethical concerns and potential emotional harm caused by such digital pranks, highlighting the need for responsible use of AI to avoid wasting emergency resources and creating community panic.
""")
response

In [ ]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")

    i +=1

TODO:
- update final prompt
- output sections using short summary
# SUGGESTED TOPICS
AI Agents And Reliability
AI Creative Industry Impact
AI Development Tools And Standards
AI Market Valuations
AI Phishing Surge
AI Security Risks
AI Workforce Impact
C2PA Image Provenance
Circular Deal Inflation
Cross-Industry AI Adoption
Crunch Lab Decentralized AI
Data Center Environmental Impact
Deepfake Video Ethics
Dell Raises AI Forecasts
EU AI Strategy
Enterprise AI Data Leakage
Enterprise AI Partnerships
Google Gemini 2.5
Healthcare AI Investments
OpenAI Platform Issues
Other
Qualcomm Acquires Arduino
SoftBank Acquires ABB Robotics
Youth Support AI Ethics

- take each summary and boil it down to 1 sentence , output correct format
- initial write sections - prompt and output json for each section asynchronously
- check and rewrite each section for format asynchronsously
- assemble sections
- do a critic loop